<a href="https://colab.research.google.com/github/paylace/ahaha/blob/main/%EC%83%9D%EC%8B%9D%EC%A0%81_%EA%B2%A9%EB%A6%AC%EB%A5%BC_%EC%9C%84%ED%95%9C_%EC%8B%9C%EB%AE%AC%EB%A0%88%EC%9D%B4%EC%85%98_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import collections
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import time
import pygame
import sys

# Matplotlib에서 한글을 깨짐 없이 표시하기 위한 설정
try:
    plt.rcParams['font.family'] = 'Malgun Gothic' # Windows
except:
    try:
        plt.rcParams['font.family'] = 'AppleGothic' # macOS
    except:
        plt.rcParams['font.family'] = 'NanumGothic' # Linux (requires installation)

# 마지막 대안으로 'sans-serif' 계열 폰트 지정
if not plt.rcParams['font.family']:
    plt.rcParams['font.family'] = ['sans-serif']
    plt.rcParams['font.sans-serif'] = ['DejaVu Sans', 'Arial'] # 일반적인 sans-serif 폰트 추가

plt.rcParams['axes.unicode_minus'] = False # 마이너스 기호 깨짐 방지


# 시뮬레이션 공통 설정 및 상수들을 모아놓은 클래스 (매직 넘버 제거)
class SimulationSettings:
    # 환경 설정
    WIDTH = 40
    HEIGHT = 40
    MAX_FOOD_PER_TILE = 100
    FOOD_REGEN_RATE = 10
    BARRIER_UPDATE_INTERVAL = 15 # 턴마다 장벽 변화 시도 주기
    BARRIER_CHANGE_PROB = 0.1    # 장벽이 사라지거나 새로 생길 확률

    # 초기 개체군 설정
    INITIAL_PREY_COUNT = 200
    INITIAL_PREDATOR_COUNT = 15
    MAX_TURNS = 300

    # 개체 형질 및 능력치 관련 상수
    BASE_MOVEMENT_SPEED_INDIVIDUAL = 1
    BASE_ENERGY_CONSUMPTION_INDIVIDUAL = 5
    MAX_ENERGY_INDIVIDUAL = 100
    MAX_AGE_INDIVIDUAL = 50 # 기본 최대 수명
    CARCASS_FOOD_VALUE_INDIVIDUAL = 10

    BASE_MOVEMENT_SPEED_PREDATOR = 3 # 포식자 기본 이동 속도 (원래 5였으나 밸런스 위해 조정)
    BASE_ENERGY_CONSUMPTION_PREY = 3
    MAX_ENERGY_PREY = 50
    MAX_AGE_PREY = 10 # 피식자 최대 수명
    CARCASS_FOOD_VALUE_PREY = 5
    PREY_REPRO_ENERGY_COST_FACTOR = 0.3 # 피식자 번식 에너지 소모 비율
    PREY_MIN_REPRO_ENERGY_FACTOR = 0.5 # 피식자 최소 번식 에너지 비율

    BASE_ENERGY_CONSUMPTION_PREDATOR = 5
    MAX_ENERGY_PREDATOR = 150
    MAX_AGE_PREDATOR = 15 # 포식자 최대 수명
    CARCASS_FOOD_VALUE_PREDATOR = 15
    PREDATOR_HUNTING_RANGE = 5
    PREDATOR_REPRO_ENERGY_COST_FACTOR = 0.4 # 포식자 번식 에너지 소모 비율
    PREDATOR_MIN_REPRO_ENERGY_FACTOR = 0.6 # 포식자 최소 번식 에너지 비율

    # 유전 및 진화 관련 상수
    LIFESPAN_POTENTIAL_AGE_BONUS_FACTOR = 0.1 # 수명 잠재력 1 증가당 최대 수명 증가 비율 (1:0%, 5:40%)
    LIFESPAN_POTENTIAL_ENERGY_COST_FACTOR = 0.5 # 수명 잠재력 1 증가당 에너지 소비 증가량
    LINEAGE_MUTATION_PROB = 0.005 # 계통 돌연변이 확률 (0.5%)
    REPRODUCTION_CYCLE_MISMATCH_PENALTY = 0.5 # 번식 주기 불일치 시 번식 성공률 감소 배율
    MIN_REPRODUCTION_CHANCE = 0.1 # 최소 번식 성공 확률

    # 형질 영향 상수
    SIZE_EFFECT_ON_MOVEMENT = 1 # 크기가 이동 속도에 미치는 영향
    MUSCLE_EFFECT_ON_MOVEMENT = 2 # 근육량이 이동 속도에 미치는 영향
    SIZE_EFFECT_ON_ENERGY_CONSUMPTION = 2 # 크기가 에너지 소비에 미치는 영향
    MUSCLE_EFFECT_ON_ENERGY_CONSUMPTION = 1 # 근육량이 에너지 소비에 미치는 영향

    PREY_SIZE_EFFECT_ON_ESCAPE = 0.05
    PREY_MUSCLE_EFFECT_ON_ESCAPE = 0.05
    PREY_CAMOUFLAGE_EFFECT_ON_ESCAPE = 0.1
    PREDATOR_SIZE_PENALTY_ON_ESCAPE = 0.03
    PREDATOR_MUSCLE_PENALTY_ON_ESCAPE = 0.04

    PRED_SIZE_EFFECT_ON_HUNT = 0.05
    PRED_MUSCLE_EFFECT_ON_HUNT = 0.04
    PRED_CAMOUFLAGE_EFFECT_ON_HUNT = 0.1
    PREY_SIZE_PENALTY_ON_HUNT = 0.05
    PREY_MUSCLE_PENALTY_ON_HUNT = 0.06
    PREY_CAMOUFLAGE_PENALTY_ON_HUNT = 0.1

    PREY_FOOD_INTAKE_BASE_CAPACITY = 5
    PREY_FOOD_INTAKE_SIZE_BONUS = 2
    PREY_RESOURCE_PREF_BONUS = 8 # 자원 선호도 일치 시 추가 먹이 획득량

    PREY_REPRO_FOOD_BONUS_FACTOR = 0.8
    PREDATOR_REPRO_ENERGY_BONUS_FACTOR = 0.5

    # 유전적 유사성 관련 상수
    GENETIC_SIMILARITY_WEIGHTS = { # 각 형질의 유사성 점수 계산 시 가중치
        'size': 0.15,
        'muscle_mass': 0.15,
        'color': 0.2,
        'reproduction_cycle': 0.15,
        'offspring_count_base': 0.1,
        'resource_preference': 0.15,
        'lifespan_potential': 0.1,
        'lineage': 0.2,
        'mate_pref_color': 0.05, # 새로운 형질 가중치
        'mate_pref_lineage': 0.05, # 새로운 형질 가중치
        'foraging_strategy': 0.05, # 새로운 형질 가중치
        'hunting_strategy': 0.05, # 새로운 형질 가중치
        'learning_rate': 0.05 # 새로운 형질 가중치
    }
    GENETIC_SIMILARITY_THRESHOLD = 0.7 # 번식 가능 최소 유전적 유사성
    GENETIC_SIMILARITY_PENALTY_FACTOR = 0.5 # 유사성 낮을 때 성공률 감소 배율

    # 짝짓기 선호도 관련 상수 (새로 추가)
    MATE_PREF_COLOR_TYPES = ['Red', 'Blue', 'Purple', 'Any'] # 선호 색상 유형
    MATE_PREF_LINEAGE_TYPES = ['Same', 'Different', 'Any'] # 선호 계통 유형 ('Same' = 자신과 같은 계통, 'Different' = 다른 계통)

    MATE_PREF_COLOR_MATCH_BONUS = 0.2 # 색상 선호도 일치 시 매력도 점수 보너스
    MATE_PREF_LINEAGE_MATCH_BONUS = 0.3 # 계통 선호도 일치 시 매력도 점수 보너스
    MATE_PREF_DIST_PENALTY_FACTOR = 0.05 # 거리당 매력도 감소 (턴당 1 감소)

    # 종분화 관련 새로운 상수 추가 (핵심 변경 부분)
    # 번식 성공을 위한 최소 호환성 점수 (이 미만은 번식 불가)
    MIN_REPRODUCTION_COMPATIBILITY = 0.5 # 0.0 (항상 가능) ~ 1.0 (완벽 일치만 가능) 사이
    # 형질 불일치에 따른 번식 성공률 감소율 (호환성 점수 직접 반영)
    TRAIT_DIVERGENCE_PENALTY_RATE = 1.0 # 1.0이면 호환성 점수에 비례하여 번식 성공률 결정

    # 자원 이용 방식 특화 관련 상수 (새로 추가)
    FORAGING_STRATEGIES = ['normal', 'wide_range', 'stealth'] # 피식자 먹이 탐색 전략
    PREY_WIDE_RANGE_MOVE_BONUS = 1 # 'wide_range' 전략 피식자 이동 스텝 보너스
    PREY_WIDE_RANGE_ENERGY_COST_MULTIPLIER = 1.2 # 'wide_range' 에너지 소비 배율
    PREY_STEALTH_ESCAPE_BONUS = 0.15 # 'stealth' 피식자 도주 확률 보너스
    PREY_STEALTH_ENERGY_COST_MULTIPLIER = 0.8 # 'stealth' 에너지 소비 배율

    HUNTING_STRATEGIES = ['pursuit', 'ambush'] # 포식자 사냥 전략
    PREDATOR_AMBUSH_HUNT_SUCCESS_BONUS = 0.2 # 'ambush' 포식자 사냥 성공 확률 보너스 (매복 성공 시)
    PREDATOR_AMBUSH_MIN_HUNTING_RANGE = 2 # 'ambush' 포식자가 매복 사냥 보너스를 받는 최소 거리
    PREDATOR_AMBUSH_MOVEMENT_REDUCTION = 0.5 # 'ambush' 포식자 이동 속도 감소 배율

    # 학습 능력 관련 상수 (새로 추가)
    LEARNING_RATE_DECAY = 0.05 # 매 턴마다 학습된 점수가 감소하는 비율 (기억 소멸)
    LEARNING_BONUS_FACTOR = 0.2 # 성공 시 학습 점수 증가 비율
    LEARNING_PENALTY_FACTOR = 0.05 # 실패 시 학습 점수 감소 비율
    LEARNED_SCORE_MIN = -1.0 # 학습된 점수의 최솟값
    LEARNED_SCORE_MAX = 1.0 # 학습된 점수의 최댓값

    CELL_SIZE = 15 # 각 그리드 셀의 크기 (픽셀). 이 값으로 화면 전체 크기가 결정됩니다.
    INFO_PANEL_WIDTH = 250 # 정보 패널의 너비 (현재는 사용되지 않지만 확장성을 위해 유지)
    FPS = 10 # 초당 프레임 수 (시뮬레이션 턴이 진행되는 속도). 높을수록 시뮬레이션이 빠르게 보입니다.

# Environment 클래스 정의
class Environment:
    def __init__(self, width, height, max_food_per_tile, food_regen_rate):
        self.width = width
        self.height = height
        self.max_food_per_tile = max_food_per_tile
        self.food_regen_rate = food_regen_rate
        self.grid, self.barrier_map = self._initialize_grid() # 각 칸의 먹이량과 환경 색상, 장벽 정보 모두 저장
        # 현재 위치에 어떤 개체들이 있는지 빠르게 찾기 위한 딕셔너리
        self.occupancy_map = collections.defaultdict(set)
        self.simulation_ref = None # Simulation 인스턴스에 대한 참조, __init__ 외부에서 설정됩니다.
        self.current_turn = 0 # 현재 턴 정보 (Simulation에서 업데이트)

    def _initialize_grid(self):
        # 환경 초기화 시 지역별 환경 색상 선호도를 설정
        self.environment_colors = ['Red', 'Blue', 'Purple']
        grid = {}
        barrier_map = {} # 장벽 정보를 저장할 맵

        # 지리적 격리: 중앙에 세로 장벽 생성
        barrier_column_x = self.width // 2

        # 지역별 환경 색상 선호도 설정
        # 왼쪽 지역은 'Red'를 선호하고, 오른쪽 지역은 'Blue'를 선호
        # 장벽의 위치에 따라 지역 구분
        left_region_preferred_color = 'Red'
        right_region_preferred_color = 'Blue'

        # 선호 색상이 나올 확률
        color_preference_prob = 0.7

        for x in range(self.width):
            for y in range(self.height):
                initial_food = self.max_food_per_tile / 2

                # 지역에 따라 환경 색상 설정
                if x < barrier_column_x: # 왼쪽 지역
                    env_color = random.choices([left_region_preferred_color] + [c for c in self.environment_colors if c != left_region_preferred_color],
                                               weights=[color_preference_prob, (1-color_preference_prob)/2, (1-color_preference_prob)/2])[0]
                else: # 오른쪽 지역 (장벽 포함)
                    env_color = random.choices([right_region_preferred_color] + [c for c in self.environment_colors if c != right_region_preferred_color],
                                               weights=[color_preference_prob, (1-color_preference_prob)/2, (1-color_preference_prob)/2])[0]

                grid[(x, y)] = {'food_amount': initial_food, 'color': env_color}

                # 장벽 설정
                if x == barrier_column_x:
                    barrier_map[(x, y)] = True # True는 장벽을 의미
                else:
                    barrier_map[(x, y)] = False # False는 장벽이 없음을 의미

        return grid, barrier_map

    def update_food(self):
        # 각 칸의 먹이량을 재생성 속도에 따라 증가시키고 최대치를 넘지 않도록 합니다.
        for pos in self.grid:
            self.grid[pos]['food_amount'] = min(self.max_food_per_tile,
                                                 self.grid[pos]['food_amount'] + self.food_regen_rate)

    def get_food_amount(self, x, y):
        # 특정 위치의 먹이량을 반환합니다.
        return self.grid.get((x, y), {'food_amount': 0})['food_amount']

    def consume_food(self, x, y, amount):
        # 특정 위치의 먹이를 소비하고, 실제로 소비된 양을 반환합니다.
        current_food = self.grid.get((x, y), {'food_amount': 0})['food_amount']
        consumed = min(current_food, amount)
        self.grid[(x, y)]['food_amount'] -= consumed
        return consumed

    def add_food_from_carcass(self, x, y, amount):
        # 죽은 개체의 시체로부터 먹이를 환경에 추가합니다.
        self.grid[(x, y)]['food_amount'] = min(self.max_food_per_tile,
                                                 self.grid[(x, y)]['food_amount'] + amount)

    def get_environment_color(self, x, y):
        # 특정 위치의 환경 색상을 반환합니다.
        return self.grid.get((x, y), {'color': None})['color']

    def is_valid_position(self, x, y):
        # 주어진 좌표가 환경 그리드 내의 유효한 위치인지 확인합니다.
        return 0 <= x < self.width and 0 <= y < self.height

    def is_position_empty(self, x, y):
        # 특정 위치가 비어있는지 (아무 개체도 없는지) 확인합니다.
        return not self.occupancy_map[(x, y)]

    def is_barrier(self, x, y):
        # 주어진 좌표가 장벽인지 확인합니다.
        return self.barrier_map.get((x, y), False) # 기본값 False (장벽 아님)

    def add_individual_to_occupancy_map(self, individual):
        # 개체를 점유 맵에 추가하여 해당 위치에 개체가 있음을 기록합니다.
        self.occupancy_map[(individual.x, individual.y)].add(individual.id)

    def remove_individual_from_occupancy_map(self, individual):
        # 개체를 점유 맵에서 제거하여 해당 위치에 개체가 없음을 기록합니다.
        if individual.id in self.occupancy_map[(individual.x, individual.y)]:
            self.occupancy_map[(individual.x, individual.y)].remove(individual.id)

    def update_barriers(self, turn, update_interval=SimulationSettings.BARRIER_UPDATE_INTERVAL,
                         barrier_change_prob=SimulationSettings.BARRIER_CHANGE_PROB):
        """
        일정 턴마다 확률적으로 장벽을 업데이트합니다.
        기존 장벽이 사라지거나 새로운 장벽이 생성될 수 있습니다.
        """
        if turn == 0: # 턴 0에서는 장벽을 움직이지 않습니다.
            return

        if turn % update_interval == 0:
            # 기존 장벽 제거 시도
            barriers_to_remove = [(x, y) for (x, y), is_barrier in self.barrier_map.items() if is_barrier]
            random.shuffle(barriers_to_remove)
            removed_count = 0
            for x, y in barriers_to_remove:
                if random.random() < barrier_change_prob:
                    self.barrier_map[(x, y)] = False
                    removed_count += 1

            # 새로운 장벽 생성 시도
            # 기존 장벽이 아닌, 개체가 없는 빈 타일에만 생성합니다.
            empty_tiles = [(x, y) for x in range(self.width) for y in range(self.height)
                           if not self.is_barrier(x, y) and not self.occupancy_map[(x,y)]]
            random.shuffle(empty_tiles)
            added_count = 0
            for x, y in empty_tiles:
                if random.random() < barrier_change_prob:
                    self.barrier_map[(x, y)] = True
                    added_count += 1

            # 장벽이 하나도 없는 경우를 대비하여 최소 1개는 유지하거나 생성 시도 (선택 사항)
            if not any(self.barrier_map.values()):
                if empty_tiles:
                    bx, by = random.choice(empty_tiles)
                    self.barrier_map[(bx, by)] = True


# Individual 클래스 정의 (Prey와 Predator의 부모 클래스)
class Individual:
    _next_id = 0 # 다음 개체에 부여할 고유 ID

    def __init__(self, env, x, y, age=0, energy=0, parent_id=None, birth_turn=None):
        self.id = Individual._next_id
        Individual._next_id += 1
        self.env = env
        self.x = x
        self.y = y
        self.age = age
        self.energy = energy
        self.is_alive = True
        self.parent_id = parent_id
        self.birth_turn = birth_turn if birth_turn is not None else self.env.current_turn

        # 개체 생성 시점에 위치 정보를 occupancy_map에 추가합니다.
        self.env.add_individual_to_occupancy_map(self)

        # 개체의 초기 형질(phenotype)을 무작위로 생성합니다.
        self.size = self._generate_random_trait_value('size')
        self.muscle_mass = self._generate_random_trait_value('muscle_mass')
        self.color = self._generate_random_trait_value('color')
        self.lifespan_potential = self._generate_random_trait_value('lifespan_potential')
        self.reproduction_cycle = self._generate_random_trait_value('reproduction_cycle', self.__class__.__name__)
        self.offspring_count_base = self._generate_random_trait_value('offspring_count_base')
        self.lineage = self._generate_random_trait_value('lineage')
        self.resource_preference = self._generate_random_trait_value('resource_preference')

        # 새로운 형질: 짝짓기 선호도
        self.mate_pref_color = self._generate_random_trait_value('mate_pref_color')
        self.mate_pref_lineage = self._generate_random_trait_value('mate_pref_lineage')

        # 새로운 형질: 자원 이용 방식 특화 (종에 따라 다르게 설정)
        if isinstance(self, Prey):
            self.foraging_strategy = self._generate_random_trait_value('foraging_strategy')
            self.hunting_strategy = None # 포식자만 가짐
        elif isinstance(self, Predator):
            self.hunting_strategy = self._generate_random_trait_value('hunting_strategy')
            self.foraging_strategy = None # 피식자만 가짐
        else:
            self.foraging_strategy = None
            self.hunting_strategy = None

        # 새로운 형질: 학습률
        self.learning_rate = self._generate_random_trait_value('learning_rate')

        # 학습된 선호도/전략을 저장할 딕셔너리 (초기값은 유전적 선호도 반영)
        self.learned_resource_pref_scores = self._initialize_learned_resource_pref()
        self.learned_mating_pref_scores = self._initialize_learned_mating_pref()

        if isinstance(self, Prey):
            self.learned_foraging_strategy_scores = self._initialize_learned_foraging_strategy()
            self.learned_hunting_strategy_scores = {}
        elif isinstance(self, Predator):
            self.learned_hunting_strategy_scores = self._initialize_learned_hunting_strategy()
            self.learned_foraging_strategy_scores = {}
        else:
            self.learned_foraging_strategy_scores = {}
            self.learned_hunting_strategy_scores = {}


        # 생성된 형질에 기반하여 유전자형(genotype)을 초기화합니다.
        self.genotype = self._generate_genotype_from_phenotype()

        # 기본 능력치 설정
        self.base_movement_speed = SimulationSettings.BASE_MOVEMENT_SPEED_INDIVIDUAL
        self.base_energy_consumption = SimulationSettings.BASE_ENERGY_CONSUMPTION_INDIVIDUAL
        self.max_energy = SimulationSettings.MAX_ENERGY_INDIVIDUAL
        self.max_age = SimulationSettings.MAX_AGE_INDIVIDUAL
        self.carcass_food_value = SimulationSettings.CARCASS_FOOD_VALUE_INDIVIDUAL

    def _initialize_learned_resource_pref(self):
        scores = {color: 0.0 for color in self.env.environment_colors}
        # 유전적 선호도를 초기 학습 점수에 반영
        if self.resource_preference in scores:
            scores[self.resource_preference] = 0.1 # 초기 선호도에 약간의 긍정 점수 부여
        return scores

    def _initialize_learned_mating_pref(self):
        scores = {f'color_{c}': 0.0 for c in SimulationSettings.MATE_PREF_COLOR_TYPES if c != 'Any'}
        scores['color_Any'] = 0.0 # Any는 별도 처리
        scores.update({f'lineage_{l}': 0.0 for l in SimulationSettings.MATE_PREF_LINEAGE_TYPES if l != 'Any'})
        scores['lineage_Any'] = 0.0 # Any는 별도 처리

        if self.mate_pref_color == 'Any':
            scores['color_Any'] = 0.1
        elif self.mate_pref_color in scores:
            scores[f'color_{self.mate_pref_color}'] = 0.1

        if self.mate_pref_lineage == 'Any':
            scores['lineage_Any'] = 0.1
        elif self.mate_pref_lineage in scores:
            scores[f'lineage_{self.mate_pref_lineage}'] = 0.1
        return scores

    def _initialize_learned_foraging_strategy(self):
        scores = {s: 0.0 for s in SimulationSettings.FORAGING_STRATEGIES}
        if self.foraging_strategy in scores:
            scores[self.foraging_strategy] = 0.1
        return scores

    def _initialize_learned_hunting_strategy(self):
        scores = {s: 0.0 for s in SimulationSettings.HUNTING_STRATEGIES}
        if self.hunting_strategy in scores:
            scores[self.hunting_strategy] = 0.1
        return scores


    def _generate_random_trait_value(self, trait_name, species_type=None):
        # 각 형질에 대한 무작위 초기값을 반환합니다.
        if trait_name == 'size':
            return random.randint(1, 5)
        elif trait_name == 'muscle_mass':
            return random.randint(1, 5)
        elif trait_name == 'color':
            return random.choice(['Red', 'Blue', 'Purple'])
        elif trait_name == 'reproduction_cycle':
            if species_type == 'Prey':
                if self.lifespan_potential == 1:
                    return random.choices([1, 2, 4], weights=[0.7, 0.2, 0.1], k=1)[0]
                elif self.lifespan_potential == 2:
                    return random.choices([1, 2, 4], weights=[0.4, 0.4, 0.2], k=1)[0]
                elif self.lifespan_potential == 3:
                    return random.choices([1, 2, 4], weights=[0.2, 0.6, 0.2], k=1)[0]
                elif self.lifespan_potential == 4:
                    return random.choices([1, 2, 4], weights=[0.1, 0.4, 0.5], k=1)[0]
                elif self.lifespan_potential == 5:
                    return random.choices([1, 2, 4], weights=[0.05, 0.15, 0.8], k=1)[0]
                else:
                    return random.choice([1, 2, 4])
            elif species_type == 'Predator':
                if self.lifespan_potential == 1:
                    return random.choices([3, 4, 5], weights=[0.7, 0.2, 0.1], k=1)[0]
                elif self.lifespan_potential == 2:
                    return random.choices([3, 4, 5], weights=[0.4, 0.4, 0.2], k=1)[0]
                elif self.lifespan_potential == 3:
                    return random.choices([3, 4, 5], weights=[0.2, 0.6, 0.2], k=1)[0]
                elif self.lifespan_potential == 4:
                    return random.choices([3, 4, 5], weights=[0.1, 0.4, 0.5], k=1)[0]
                elif self.lifespan_potential == 5:
                    return random.choices([3, 4, 5], weights=[0.05, 0.15, 0.8], k=1)[0]
                else:
                    return random.choice([3, 4, 5])
            else:
                return random.choice([5, 8, 12])
        elif trait_name == 'resource_preference':
            return random.choice(['Red', 'Blue', 'Purple'])
        elif trait_name == 'lifespan_potential':
            return random.randint(1, 5)
        elif trait_name == 'offspring_count_base':
            return random.randint(1, 3)
        elif trait_name == 'lineage':
            return 'A'
        # 새로운 형질: 짝짓기 선호도
        elif trait_name == 'mate_pref_color':
            return random.choice(SimulationSettings.MATE_PREF_COLOR_TYPES)
        elif trait_name == 'mate_pref_lineage':
            return random.choice(SimulationSettings.MATE_PREF_LINEAGE_TYPES)
        # 새로운 형질: 자원 이용 방식 특화
        elif trait_name == 'foraging_strategy':
            return random.choice(SimulationSettings.FORAGING_STRATEGIES)
        elif trait_name == 'hunting_strategy':
            return random.choice(SimulationSettings.HUNTING_STRATEGIES)
        # 새로운 형질: 학습률
        elif trait_name == 'learning_rate':
            return random.uniform(0.1, 1.0) # 0.1에서 1.0 사이의 실수 값
        return 0

    def _generate_genotype_from_phenotype(self):
        # 현재 개체의 형질에 기반하여 유전자형을 생성합니다.
        genotype = {}
        # 색상 유전자형 (A: Red, B: Blue, A+B: Purple)
        if self.color == 'Red':
            genotype['color'] = ['A', 'A']
        elif self.color == 'Blue':
            genotype['color'] = ['B', 'B']
        else: # Purple
            genotype['color'] = sorted(random.sample(['A', 'B'], 2))

        # 크기 유전자형 (S1이 우성)
        num_dominant_s = self.size - 1
        s_alleles_pool = ['S1'] * num_dominant_s + ['s1'] * (4 - num_dominant_s)
        random.shuffle(s_alleles_pool)
        genotype['size'] = (sorted(s_alleles_pool[:2]), sorted(s_alleles_pool[2:]))

        # 근육량 유전자형 (M1이 우성)
        num_dominant_m = self.muscle_mass - 1
        m_alleles_pool = ['M1'] * num_dominant_m + ['m1'] * (4 - num_dominant_m)
        random.shuffle(m_alleles_pool)
        genotype['muscle_mass'] = (sorted(m_alleles_pool[:2]), sorted(m_alleles_pool[2:]))

        # 번식 주기 유전자형 (C: 짧은 주기, c: 긴 주기)
        if self.reproduction_cycle == 1 or self.reproduction_cycle == 3:
            genotype['reproduction_cycle'] = ['C', 'C']
        elif self.reproduction_cycle == 2 or self.reproduction_cycle == 4:
            genotype['reproduction_cycle'] = sorted(random.sample(['C', 'c'], 2))
        else:
            genotype['reproduction_cycle'] = ['c', 'c']

        # 자손 수 기본값 유전자형 (O: 많음, o: 적음)
        if self.offspring_count_base == 3:
            genotype['offspring_count_base'] = ['O', 'O']
        elif self.offspring_count_base == 2:
            genotype['offspring_count_base'] = sorted(random.sample(['O', 'o'], 2))
        else:
            genotype['offspring_count_base'] = ['o', 'o']

        genotype['lineage'] = [self.lineage, self.lineage]
        genotype['resource_preference'] = [self.resource_preference, self.resource_preference]
        genotype['lifespan_potential'] = [str(self.lifespan_potential), str(self.lifespan_potential)]

        # 새로운 형질: 짝짓기 선호도 유전자형
        genotype['mate_pref_color'] = [self.mate_pref_color, self.mate_pref_color]
        genotype['mate_pref_lineage'] = [self.mate_pref_lineage, self.mate_pref_lineage]

        # 새로운 형질: 자원 이용 방식 특화 유전자형 (종에 따라)
        if isinstance(self, Prey):
            genotype['foraging_strategy'] = [self.foraging_strategy, self.foraging_strategy]
        elif isinstance(self, Predator):
            genotype['hunting_strategy'] = [self.hunting_strategy, self.hunting_strategy]

        # 새로운 형질: 학습률 유전자형 (값을 문자열로 변환하여 저장)
        genotype['learning_rate'] = [str(self.learning_rate), str(self.learning_rate)]

        return genotype

    def calculate_phenotype_from_genotype(self, genotype_data):
        # 주어진 유전자형 데이터로부터 형질을 계산하여 반환합니다.
        phenotype = {}

        color_alleles = genotype_data.get('color', [])
        if 'A' in color_alleles and 'B' in color_alleles:
            phenotype['color'] = 'Purple'
        elif 'A' in color_alleles:
            phenotype['color'] = 'Red'
        else:
            phenotype['color'] = 'Blue'

        size_alleles_pair1, size_alleles_pair2 = genotype_data.get('size', ([],[]))
        dominant_s_count = sum(1 for allele in size_alleles_pair1 if allele.isupper()) + \
                             sum(1 for allele in size_alleles_pair2 if allele.isupper())
        phenotype['size'] = 1 + dominant_s_count

        muscle_alleles_pair1, muscle_alleles_pair2 = genotype_data.get('muscle_mass', ([],[]))
        dominant_m_count = sum(1 for allele in muscle_alleles_pair1 if allele.isupper()) + \
                             sum(1 for allele in muscle_alleles_pair2 if allele.isupper())
        phenotype['muscle_mass'] = 1 + dominant_m_count

        repro_cycle_alleles = genotype_data.get('reproduction_cycle', [])
        if 'C' in repro_cycle_alleles and 'c' in repro_cycle_alleles:
            phenotype['reproduction_cycle'] = 2 if isinstance(self, Prey) else 4
        elif 'C' in repro_cycle_alleles:
            phenotype['reproduction_cycle'] = 1 if isinstance(self, Prey) else 3
        else:
            phenotype['reproduction_cycle'] = 4 if isinstance(self, Prey) else 5

        offspring_alleles = genotype_data.get('offspring_count_base', [])
        if 'O' in offspring_alleles and 'o' in offspring_alleles:
            phenotype['offspring_count_base'] = 2
        elif 'O' in offspring_alleles:
            phenotype['offspring_count_base'] = 3
        else:
            phenotype['offspring_count_base'] = 1

        lineage_alleles = genotype_data.get('lineage', [])
        phenotype['lineage'] = lineage_alleles[0] if lineage_alleles else 'A' # 기본값 처리

        resource_preference_alleles = genotype_data.get('resource_preference', [])
        phenotype['resource_preference'] = resource_preference_alleles[0] if resource_preference_alleles else 'Any'

        lifespan_potential_alleles = genotype_data.get('lifespan_potential', [])
        phenotype['lifespan_potential'] = int(lifespan_potential_alleles[0]) if lifespan_potential_alleles else 3

        # 새로운 형질: 짝짓기 선호도 형질 계산
        mate_pref_color_alleles = genotype_data.get('mate_pref_color', [])
        phenotype['mate_pref_color'] = mate_pref_color_alleles[0] if mate_pref_color_alleles else 'Any'

        mate_pref_lineage_alleles = genotype_data.get('mate_pref_lineage', [])
        phenotype['mate_pref_lineage'] = mate_pref_lineage_alleles[0] if mate_pref_lineage_alleles else 'Any'

        # 새로운 형질: 자원 이용 방식 특화 형질 계산
        if isinstance(self, Prey):
            foraging_strategy_alleles = genotype_data.get('foraging_strategy', [])
            phenotype['foraging_strategy'] = foraging_strategy_alleles[0] if foraging_strategy_alleles else 'normal'
        elif isinstance(self, Predator):
            hunting_strategy_alleles = genotype_data.get('hunting_strategy', [])
            phenotype['hunting_strategy'] = hunting_strategy_alleles[0] if hunting_strategy_alleles else 'pursuit'

        # 새로운 형질: 학습률 계산 (문자열 값을 float로 변환)
        learning_rate_alleles = genotype_data.get('learning_rate', [])
        phenotype['learning_rate'] = float(learning_rate_alleles[0]) if learning_rate_alleles else 0.5

        return phenotype

    def get_effective_movement_speed(self):
        # 개체의 유효 이동 속도를 계산합니다.
        speed = self.base_movement_speed + (self.muscle_mass * SimulationSettings.MUSCLE_EFFECT_ON_MOVEMENT) - \
                (self.size * SimulationSettings.SIZE_EFFECT_ON_MOVEMENT)

        # 자원 이용 방식 특화에 따른 이동 속도 보정 (prey/predator 클래스에서 재정의될 수 있음)
        # 'stealth' 피식자는 이동 속도 감소로 간주
        if isinstance(self, Prey) and self.foraging_strategy == 'stealth':
            speed *= SimulationSettings.PREY_STEALTH_ENERGY_COST_MULTIPLIER
        # 'ambush' 포식자는 매복 전략 이동 속도 감소
        elif isinstance(self, Predator) and self.hunting_strategy == 'ambush':
            speed *= SimulationSettings.PREDATOR_AMBUSH_MOVEMENT_REDUCTION

        return max(1, round(speed)) # 최소 속도는 1

    def get_energy_consumption(self):
        # 개체의 턴당 에너지 소비량을 계산합니다.
        consumption = self.base_energy_consumption + (self.size * SimulationSettings.SIZE_EFFECT_ON_ENERGY_CONSUMPTION) + \
                      (self.muscle_mass * SimulationSettings.MUSCLE_EFFECT_ON_ENERGY_CONSUMPTION) + \
                      (self.lifespan_potential * SimulationSettings.LIFESPAN_POTENTIAL_ENERGY_COST_FACTOR)

        # 자원 이용 방식 특화에 따른 에너지 소비 보정
        if isinstance(self, Prey):
            if self.foraging_strategy == 'wide_range':
                consumption *= SimulationSettings.PREY_WIDE_RANGE_ENERGY_COST_MULTIPLIER
            elif self.foraging_strategy == 'stealth':
                consumption *= SimulationSettings.PREY_STEALTH_ENERGY_COST_MULTIPLIER

        return consumption

    def move(self):
        # 개체를 환경 내에서 이동시킵니다. (Individual 클래스의 기본 이동 로직)
        if not self.is_alive: return

        self.env.remove_individual_from_occupancy_map(self)

        speed = self.get_effective_movement_speed()
        for _ in range(speed):
            dx, dy = random.choice([(0, 1), (0, -1), (1, 0), (-1, 0)])
            new_x, new_y = self.x + dx, self.y + dy

            if self.env.is_valid_position(new_x, new_y) and not self.env.is_barrier(new_x, new_y):
                self.x, self.y = new_x, new_y
            else:
                break

        self.env.add_individual_to_occupancy_map(self)

    def _move_towards_target(self, target_x, target_y):
        # 특정 목표 좌표로 개체를 이동시킵니다.
        if not self.is_alive: return

        self.env.remove_individual_from_occupancy_map(self)

        current_x, current_y = self.x, self.y
        speed = self.get_effective_movement_speed()

        for _ in range(speed):
            dx, dy = 0, 0
            if target_x > current_x: dx = 1
            elif target_x < current_x: dx = -1

            if target_y > current_y: dy = 1
            elif target_y < current_y: dy = -1

            if dx == 0 and dy == 0: break

            next_x, next_y = current_x + dx, current_y + dy

            if self.env.is_valid_position(next_x, next_y) and not self.env.is_barrier(next_x, next_y):
                current_x, current_y = next_x, next_y
            else:
                valid_random_moves = []
                for rdx, rdy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                    rand_x, rand_y = current_x + rdx, current_y + rdy
                    if self.env.is_valid_position(rand_x, rand_y) and not self.env.is_barrier(rand_x, rand_y):
                        valid_random_moves.append((rand_x, rand_y))

                if valid_random_moves:
                    current_x, current_y = random.choice(valid_random_moves)
                else:
                    break

        self.x, self.y = current_x, current_y
        self.x = max(0, min(self.env.width - 1, self.x))
        self.y = max(0, min(self.env.height - 1, self.y))

        self.env.add_individual_to_occupancy_map(self)


    def age_and_check_death(self):
        # 개체의 나이를 먹이고, 수명이 다했는지 확인하여 죽음을 처리합니다.
        if not self.is_alive: return False

        self.age += 1

        actual_max_age = self.max_age * (1 + (self.lifespan_potential - 1) * SimulationSettings.LIFESPAN_POTENTIAL_AGE_BONUS_FACTOR)

        if self.age >= actual_max_age:
            self.die("수명 만료")
            return True
        return False

    def die(self, reason="알 수 없음"):
        # 개체가 죽었을 때의 처리를 합니다 (생존 상태 변경, 점유 맵에서 제거, 시체 먹이 추가).
        if self.is_alive:
            self.env.simulation_ref.turn_logs[self.env.current_turn].append(f"개체 {self.id} ({self.__class__.__name__})가 턴 {self.env.current_turn}에 {reason}으로 죽었습니다. (출생 턴: {self.birth_turn})")
            self.is_alive = False
            self.env.remove_individual_from_occupancy_map(self)
            self.env.add_food_from_carcass(self.x, self.y, self.carcass_food_value)

    def _calculate_genetic_similarity(self, other_individual):
        """
        두 개체의 유전자형을 비교하여 유전적 유사성 점수(0.0~1.0)를 계산합니다.
        각 형질별 대립유전자 일치 정도를 가중치에 따라 합산합니다.
        """
        similarity_score = 0.0
        total_weight = sum(SimulationSettings.GENETIC_SIMILARITY_WEIGHTS.values())

        for trait, weight in SimulationSettings.GENETIC_SIMILARITY_WEIGHTS.items():
            # 특화 전략 형질은 종마다 다르게 존재하므로 조건부로 접근
            if (isinstance(self, Prey) and trait == 'hunting_strategy') or \
               (isinstance(self, Predator) and trait == 'foraging_strategy'):
                continue # 현재 개체 종에 없는 형질은 비교에서 제외

            gene1_alleles = self.genotype.get(trait)
            gene2_alleles = other_individual.genotype.get(trait)

            if gene1_alleles is None or gene2_alleles is None:
                continue # 유전자형에 없는 형질은 비교에서 제외

            if isinstance(gene1_alleles, tuple): # size, muscle_mass와 같이 두 쌍의 유전자를 가지는 경우
                match_count_pair1 = sum(1 for allele in gene1_alleles[0] if allele in gene2_alleles[0])
                match_count_pair2 = sum(1 for allele in gene1_alleles[1] if allele in gene2_alleles[1])
                trait_similarity = ((match_count_pair1 / len(gene1_alleles[0])) + (match_count_pair2 / len(gene1_alleles[1]))) / 2
            else: # color, reproduction_cycle, offspring_count_base, lineage, resource_preference, lifespan_potential, mate_pref_color, mate_pref_lineage, foraging_strategy, hunting_strategy, learning_rate
                # 학습률은 문자열로 저장된 float이므로, 숫자로 변환 후 차이 역으로 유사성 계산
                if trait == 'learning_rate':
                    val1 = float(gene1_alleles[0]) # 단순화: 두 유전자 값 동일하다고 가정
                    val2 = float(gene2_alleles[0])
                    # 유사성은 (1 - 절대 오차)로 계산
                    # min(val1, val2) / max(val1, val2)로 해도 됨
                    trait_similarity = 1 - abs(val1 - val2) / max(val1, val2, 0.01) # 0으로 나누는 것 방지
                else: # 나머지 단일 유전자 형질
                    matched_alleles = set(gene1_alleles) & set(gene2_alleles)
                    trait_similarity = len(matched_alleles) / 2.0

            similarity_score += trait_similarity * weight

        if total_weight == 0: return 0.0 # Division by zero 방지

        return similarity_score / total_weight

    # 새로운 메서드: 짝짓기 호환성 계산 (핵심 변경 부분)
    def _calculate_mating_compatibility(self, other_individual):
        """
        두 개체 간의 짝짓기 호환성 점수(0.0~1.0)를 계산합니다.
        생식적 격리에 중요한 형질(색상, 계통, 번식 주기)에 중점을 둡니다.
        """
        compatibility_score = 0.0
        # 종분화에 가장 직접적으로 영향을 미치는 형질들에 가중치를 더 부여
        # 이 가중치는 GENETIC_SIMILARITY_WEIGHTS와는 별개로 호환성 계산에만 사용
        compatibility_weights = {
            'color': 0.4, # 색상이 다르면 아예 번식 못하게 할 수도 있음
            'lineage': 0.3, # 계통이 다르면 번식력 저하
            'reproduction_cycle': 0.3, # 번식 주기가 맞지 않으면 번식 불가
        }
        total_comp_weight = sum(compatibility_weights.values())

        for trait, weight in compatibility_weights.items():
            gene1_alleles = self.genotype.get(trait)
            gene2_alleles = other_individual.genotype.get(trait)

            if gene1_alleles is None or gene2_alleles is None:
                continue

            if isinstance(gene1_alleles, tuple): # size, muscle_mass (여기서는 사용되지 않지만 일반성 유지)
                match_count_pair1 = sum(1 for allele in gene1_alleles[0] if allele in gene2_alleles[0])
                match_count_pair2 = sum(1 for allele in gene1_alleles[1] if allele in gene2_alleles[1])
                trait_similarity = ((match_count_pair1 / len(gene1_alleles[0])) + (match_count_pair2 / len(gene1_alleles[1]))) / 2
            else: # color, reproduction_cycle, lineage
                matched_alleles = set(gene1_alleles) & set(gene2_alleles)
                trait_similarity = len(matched_alleles) / 2.0 # 2개의 유전자 중 일치하는 것의 비율

            compatibility_score += trait_similarity * weight

        if total_comp_weight == 0: return 0.0

        final_compatibility = compatibility_score / total_comp_weight
        return final_compatibility


    def _create_child_properties(self, parent1_genotype, parent2_genotype, is_gene_based_sim):
        """
        두 부모의 유전자형을 기반으로 자손의 유전자형 및 형질 데이터를 생성합니다.
        is_gene_based_sim에 따라 유전 법칙을 따르거나 무작위로 생성합니다.
        """
        child_genotype = {}

        if is_gene_based_sim:
            child_genotype['color'] = sorted([random.choice(parent1_genotype['color']),
                                              random.choice(parent2_genotype['color'])])

            child_genotype['size'] = (
                sorted([random.choice(parent1_genotype['size'][0]), random.choice(parent2_genotype['size'][0])]),
                sorted([random.choice(parent1_genotype['size'][1]), random.choice(parent2_genotype['size'][1])])
            )

            child_genotype['muscle_mass'] = (
                sorted([random.choice(parent1_genotype['muscle_mass'][0]), random.choice(parent2_genotype['muscle_mass'][0])]),
                sorted([random.choice(parent1_genotype['muscle_mass'][1]), random.choice(parent2_genotype['muscle_mass'][1])])
            )

            child_genotype['reproduction_cycle'] = sorted([random.choice(parent1_genotype['reproduction_cycle']),
                                                           random.choice(parent2_genotype['reproduction_cycle'])])

            child_genotype['offspring_count_base'] = sorted([random.choice(parent1_genotype['offspring_count_base']),
                                                              random.choice(parent2_genotype['offspring_count_base'])])

            child_genotype['resource_preference'] = sorted([random.choice(parent1_genotype['resource_preference']),
                                                            random.choice(parent2_genotype['resource_preference'])])

            child_genotype['lifespan_potential'] = sorted([random.choice(parent1_genotype['lifespan_potential']),
                                                           random.choice(parent2_genotype['lifespan_potential'])])

            child_genotype['lineage'] = sorted([random.choice(parent1_genotype['lineage']),
                                                 random.choice(parent2_genotype['lineage'])])

            # 계통 형질의 돌연변이: 아주 낮은 확률로 새로운 계통 발생
            if random.random() < SimulationSettings.LINEAGE_MUTATION_PROB:
                current_lineage_char = ord(child_genotype['lineage'][0])
                new_lineage_char = chr(current_lineage_char + 1)
                child_genotype['lineage'] = [new_lineage_char, new_lineage_char]

            # 새로운 형질: 짝짓기 선호도 유전
            child_genotype['mate_pref_color'] = sorted([random.choice(parent1_genotype.get('mate_pref_color', ['Any'])),
                                                         random.choice(parent2_genotype.get('mate_pref_color', ['Any']))])
            child_genotype['mate_pref_lineage'] = sorted([random.choice(parent1_genotype.get('mate_pref_lineage', ['Any'])),
                                                           random.choice(parent2_genotype.get('mate_pref_lineage', ['Any']))])

            # 새로운 형질: 자원 이용 방식 특화 유전
            if isinstance(self, Prey):
                child_genotype['foraging_strategy'] = sorted([random.choice(parent1_genotype.get('foraging_strategy', ['normal'])),
                                                              random.choice(parent2_genotype.get('foraging_strategy', ['normal']))])
            elif isinstance(self, Predator):
                child_genotype['hunting_strategy'] = sorted([random.choice(parent1_genotype.get('hunting_strategy', ['pursuit'])),
                                                              random.choice(parent2_genotype.get('hunting_strategy', ['pursuit']))])

            # 새로운 형질: 학습률 유전 (부모 중 하나로부터 무작위로 선택)
            child_genotype['learning_rate'] = sorted([random.choice(parent1_genotype.get('learning_rate', ['0.5'])),
                                                      random.choice(parent2_genotype.get('learning_rate', ['0.5']))])

        else: # 무작위 시뮬레이션일 경우
            child_genotype = {
                'color': sorted([random.choice(['A', 'B']), random.choice(['A', 'B'])]),
                'size': (sorted([random.choice(['S1', 's1']), random.choice(['S1', 's1'])]),
                         sorted([random.choice(['S2', 's2']), random.choice(['S2', 's2'])])),
                'muscle_mass': (sorted([random.choice(['M1', 'm1']), random.choice(['M1', 'm1'])]),
                                 sorted([random.choice(['M2', 'm2']), random.choice(['M2', 'm2'])])),
                'reproduction_cycle': sorted([random.choice(['C', 'c']), random.choice(['C', 'c'])]),
                'offspring_count_base': sorted([random.choice(['O', 'o']), random.choice(['O', 'o'])]),
                'lineage': ['A', 'A'],
                'resource_preference': sorted([random.choice(self.env.environment_colors), random.choice(self.env.environment_colors)]),
                'lifespan_potential': sorted([str(random.randint(1, 5)), str(random.randint(1, 5))] ),
                'mate_pref_color': sorted([random.choice(SimulationSettings.MATE_PREF_COLOR_TYPES), random.choice(SimulationSettings.MATE_PREF_COLOR_TYPES)]),
                'mate_pref_lineage': sorted([random.choice(SimulationSettings.MATE_PREF_LINEAGE_TYPES), random.choice(SimulationSettings.MATE_PREF_LINEAGE_TYPES)]),
                'learning_rate': sorted([str(random.uniform(0.1, 1.0)), str(random.uniform(0.1, 1.0))]), # 무작위 학습률
            }
            if isinstance(self, Prey):
                child_genotype['foraging_strategy'] = sorted([random.choice(SimulationSettings.FORAGING_STRATEGIES), random.choice(SimulationSettings.FORAGING_STRATEGIES)])
            elif isinstance(self, Predator):
                child_genotype['hunting_strategy'] = sorted([random.choice(SimulationSettings.HUNTING_STRATEGIES), random.choice(SimulationSettings.HUNTING_STRATEGIES)])

        # 자손의 형질 계산
        child_phenotype_data = self.calculate_phenotype_from_genotype(child_genotype)

        return child_genotype, child_phenotype_data


    def process_feedback(self, feedback_type, success, context_info=None):
        """
        개체가 경험으로부터 학습하여 내부 점수를 업데이트합니다.
        :param feedback_type: 'food_acquisition', 'mating_success', 'hunting_success', 'strategy_use' 등 피드백 유형
        :param success: True (성공) 또는 False (실패)
        :param context_info: 피드백과 관련된 추가 정보 (예: 환경 색상, 파트너 색상, 사용한 전략 이름)
        """
        # 학습된 점수 업데이트
        if feedback_type == 'food_acquisition' and context_info: # context_info는 환경 색상
            target_key = context_info # 환경 색상 (예: 'Red')
            if success:
                self.learned_resource_pref_scores[target_key] += self.learning_rate * SimulationSettings.LEARNING_BONUS_FACTOR
            else:
                self.learned_resource_pref_scores[target_key] -= self.learning_rate * SimulationSettings.LEARNING_PENALTY_FACTOR
            self.learned_resource_pref_scores[target_key] = max(SimulationSettings.LEARNED_SCORE_MIN, min(SimulationSettings.LEARNED_SCORE_MAX, self.learned_resource_pref_scores[target_key]))

        elif feedback_type == 'mating_success' and context_info: # context_info는 (파트너 색상, 파트너 계통, 자손 건강 여부)
            partner_color_pref_key = f'color_{context_info["partner_color"]}' if context_info["partner_color"] != 'Any' else 'color_Any'
            partner_lineage_pref_key = f'lineage_{context_info["partner_lineage_type"]}' if context_info["partner_lineage_type"] != 'Any' else 'lineage_Any'

            # 파트너 색상에 대한 학습
            if partner_color_pref_key in self.learned_mating_pref_scores:
                if success:
                    self.learned_mating_pref_scores[partner_color_pref_key] += self.learning_rate * SimulationSettings.LEARNING_BONUS_FACTOR
                else:
                    self.learned_mating_pref_scores[partner_color_pref_key] -= self.learning_rate * SimulationSettings.LEARNING_PENALTY_FACTOR
                self.learned_mating_pref_scores[partner_color_pref_key] = max(SimulationSettings.LEARNED_SCORE_MIN, min(SimulationSettings.LEARNED_SCORE_MAX, self.learned_mating_pref_scores[partner_color_pref_key]))

            # 파트너 계통에 대한 학습
            if partner_lineage_pref_key in self.learned_mating_pref_scores:
                if success:
                    self.learned_mating_pref_scores[partner_lineage_pref_key] += self.learning_rate * SimulationSettings.LEARNING_BONUS_FACTOR
                else:
                    self.learned_mating_pref_scores[partner_lineage_pref_key] -= self.learning_rate * SimulationSettings.LEARNING_PENALTY_FACTOR
                self.learned_mating_pref_scores[partner_lineage_pref_key] = max(SimulationSettings.LEARNED_SCORE_MIN, min(SimulationSettings.LEARNED_SCORE_MAX, self.learned_mating_pref_scores[partner_lineage_pref_key]))

        elif feedback_type == 'foraging_strategy_use' and context_info: # context_info는 사용된 전략 이름
            target_key = context_info # 전략 이름 (예: 'normal')
            if target_key in self.learned_foraging_strategy_scores:
                if success:
                    self.learned_foraging_strategy_scores[target_key] += self.learning_rate * SimulationSettings.LEARNING_BONUS_FACTOR
                else:
                    self.learned_foraging_strategy_scores[target_key] -= self.learning_rate * SimulationSettings.LEARNING_PENALTY_FACTOR
                self.learned_foraging_strategy_scores[target_key] = max(SimulationSettings.LEARNED_SCORE_MIN, min(SimulationSettings.LEARNED_SCORE_MAX, self.learned_foraging_strategy_scores[target_key]))

        elif feedback_type == 'hunting_strategy_use' and context_info: # context_info는 사용된 전략 이름
            target_key = context_info # 전략 이름 (예: 'pursuit')
            if target_key in self.learned_hunting_strategy_scores:
                if success:
                    self.learned_hunting_strategy_scores[target_key] += self.learning_rate * SimulationSettings.LEARNING_BONUS_FACTOR
                else:
                    self.learned_hunting_strategy_scores[target_key] -= self.learning_rate * SimulationSettings.LEARNING_PENALTY_FACTOR
                self.learned_hunting_strategy_scores[target_key] = max(SimulationSettings.LEARNED_SCORE_MIN, min(SimulationSettings.LEARNED_SCORE_MAX, self.learned_hunting_strategy_scores[target_key]))

        # 기억 소멸 (Memory Decay): 매 턴마다 모든 학습된 점수를 기본값(0)으로 회귀시킴
        for scores_dict in [self.learned_resource_pref_scores, self.learned_mating_pref_scores,
                            self.learned_foraging_strategy_scores, self.learned_hunting_strategy_scores]:
            for key in scores_dict:
                scores_dict[key] *= (1 - SimulationSettings.LEARNING_RATE_DECAY)
                # 점수 범위를 유지
                scores_dict[key] = max(SimulationSettings.LEARNED_SCORE_MIN, min(SimulationSettings.LEARNED_SCORE_MAX, scores_dict[key]))


    def get_color_match_score(self):
        # 현재 위치의 환경 색상과 개체 자신의 색상이 얼마나 일치하는지 점수를 반환합니다.
        env_color = self.env.get_environment_color(self.x, self.y)
        if self.color == env_color:
            return 1.0
        elif (self.color == 'Purple' and (env_color == 'Red' or env_color == 'Blue')) or \
             ((self.color == 'Red' or self.color == 'Blue') and env_color == 'Purple'):
            return 0.5
        else:
            return 0.0

    def get_mating_attractiveness(self, partner_individual):
        """
        이 개체(self)가 다른 개체(partner_individual)를 짝짓기 파트너로서 얼마나 매력적으로 느끼는지 점수(0.0~1.0)를 반환합니다.
        유전적 선호도와 학습된 선호도를 모두 고려합니다.
        """
        attractiveness_score = 0.0

        # 유전적 색상 선호도
        if self.mate_pref_color == 'Any':
            attractiveness_score += 0.3
        elif self.mate_pref_color == partner_individual.color:
            attractiveness_score += 0.5 + SimulationSettings.MATE_PREF_COLOR_MATCH_BONUS
        elif (self.mate_pref_color == 'Purple' and (partner_individual.color == 'Red' or partner_individual.color == 'Blue')) or \
             ((self.mate_pref_color == 'Red' or self.mate_pref_color == 'Blue') and partner_individual.color == 'Purple'):
            attractiveness_score += 0.25

        # 학습된 색상 선호도 추가
        learned_color_pref_key = f'color_{partner_individual.color}' if partner_individual.color != 'Any' else 'color_Any'
        if learned_color_pref_key in self.learned_mating_pref_scores:
            attractiveness_score += self.learned_mating_pref_scores[learned_color_pref_key] * 0.2 # 학습 점수 가중치 (조절 가능)

        # 유전적 계통 선호도
        partner_lineage_type = 'Same' if self.lineage == partner_individual.lineage else 'Different'
        if self.mate_pref_lineage == 'Any':
            attractiveness_score += 0.3
        elif self.mate_pref_lineage == partner_lineage_type:
            attractiveness_score += 0.5 + SimulationSettings.MATE_PREF_LINEAGE_MATCH_BONUS

        # 학습된 계통 선호도 추가
        learned_lineage_pref_key = f'lineage_{partner_lineage_type}' if partner_lineage_type != 'Any' else 'lineage_Any'
        if learned_lineage_pref_key in self.learned_mating_pref_scores:
            attractiveness_score += self.learned_mating_pref_scores[learned_lineage_pref_key] * 0.3 # 학습 점수 가중치 (조절 가능)


        # 거리에 따른 매력도 감소 (가까울수록 좋음)
        dist = abs(self.x - partner_individual.x) + abs(self.y - partner_individual.y)
        attractiveness_score -= dist * SimulationSettings.MATE_PREF_DIST_PENALTY_FACTOR

        return max(0.0, min(1.0, attractiveness_score)) # 점수는 0.0에서 1.0 사이

    def __repr__(self):
        # 개체 정보를 문자열로 표현합니다.
        actual_max_age = self.max_age * (1 + (self.lifespan_potential - 1) * SimulationSettings.LIFESPAN_POTENTIAL_AGE_BONUS_FACTOR)

        # 자원 이용 방식 특화 형질을 __repr__에 포함
        strategy_info = ""
        if isinstance(self, Prey) and self.foraging_strategy:
            strategy_info = f", ForageStrat:{self.foraging_strategy}"
        elif isinstance(self, Predator) and self.hunting_strategy:
            strategy_info = f", HuntStrat:{self.hunting_strategy}"

        return f"{self.__class__.__name__}(ID:{self.id}, Pos:({self.x},{self.y}), Age:{self.age}/{int(actual_max_age)}, Energy:{int(self.energy)}, Size:{self.size}, Muscle:{self.muscle_mass}, Color:{self.color}, ReproCycle:{self.reproduction_cycle}, OffspringBase:{self.offspring_count_base}, Lineage:{self.lineage}, ResPref:{self.resource_preference}, LifespanPot:{self.lifespan_potential}, MatePrefColor:{self.mate_pref_color}, MatePrefLineage:{self.mate_pref_lineage}{strategy_info}, LearningRate:{self.learning_rate:.2f}, BirthTurn:{self.birth_turn})"


class Prey(Individual):
    def __init__(self, env, x, y, age=0, energy=SimulationSettings.MAX_ENERGY_PREY / 2, parent_id=None, birth_turn=None):
        super().__init__(env, x, y, age, energy, parent_id, birth_turn=birth_turn)
        self.max_energy = SimulationSettings.MAX_ENERGY_PREY
        self.base_energy_consumption = SimulationSettings.BASE_ENERGY_CONSUMPTION_PREY
        self.max_age = SimulationSettings.MAX_AGE_PREY
        self.carcass_food_value = SimulationSettings.CARCASS_FOOD_VALUE_PREY

    def live_a_turn(self, all_prey):
        # 한 턴 동안 피식자의 행동 (나이 증가, 에너지 소비, 먹이 섭취, 이동)을 처리합니다.
        if not self.is_alive: return

        if self.age_and_check_death(): return

        self.energy -= self.get_energy_consumption()
        if self.energy <= 0:
            self.die("굶주림")
            return

        # 번식 우선순위 로직
        if self.age >= self.reproduction_cycle - 1 and self.energy >= self.max_energy * SimulationSettings.PREY_MIN_REPRO_ENERGY_FACTOR:
            best_partner_pos = None
            max_attractiveness = -1.0 # 짝짓기 매력도 점수 초기화

            for other_prey_id in all_prey:
                other_prey = all_prey[other_prey_id]
                if other_prey.is_alive and other_prey.id != self.id and \
                   other_prey.age >= other_prey.reproduction_cycle - 1:

                    attractiveness = self.get_mating_attractiveness(other_prey) # 매력도 계산
                    if attractiveness > max_attractiveness:
                        max_attractiveness = attractiveness
                        best_partner_pos = (other_prey.x, other_prey.y)

            if best_partner_pos and max_attractiveness > 0: # 매력적인 파트너를 찾았고 매력도가 0보다 클 때 이동
                self._move_towards_target(best_partner_pos[0], best_partner_pos[1])
            else: # 파트너를 찾지 못했거나 매력적이지 않으면 먹이 탐색
                self._perform_food_seeking_move()
        else:
            self._perform_food_seeking_move()

        # 현재 위치에서 먹이 섭취
        initial_energy = self.energy
        food_consumed = self.env.consume_food(self.x, self.y, self.get_food_intake_capacity())
        self.energy = min(self.max_energy, self.energy + food_consumed)

        # 먹이 획득에 대한 피드백 처리
        self.process_feedback('food_acquisition', food_consumed > 0, self.env.get_environment_color(self.x, self.y))


    def _perform_food_seeking_move(self):
        # 피식자 개체를 환경 내에서 이동시킵니다.
        if not self.is_alive: return

        self.env.remove_individual_from_occupancy_map(self)

        current_x, current_y = self.x, self.y
        speed = self.get_effective_movement_speed()

        # 'wide_range' 전략 적용
        if self.foraging_strategy == 'wide_range':
            speed += SimulationSettings.PREY_WIDE_RANGE_MOVE_BONUS # 이동 스텝 보너스

        # 학습된 전략 점수에 따라 이동 방향 선택에 영향 (추가 구현)
        # 현재는 foraging_strategy가 고정된 형질이므로, 행동 자체에 영향을 줌.
        # 만약 foraging_strategy가 매 턴 선택될 수 있다면, learned_foraging_strategy_scores 사용.

        chosen_strategy = self.foraging_strategy # 현재는 고정된 전략 사용

        for _ in range(speed):
            potential_target_positions = []
            max_effective_score = -float('inf') # 먹이량 + 학습된 선호도 점수

            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    if dx == 0 and dy == 0: continue

                    nx, ny = current_x + dx, current_y + dy
                    if self.env.is_valid_position(nx, ny) and not self.env.is_barrier(nx, ny):
                        food_amount = self.env.get_food_amount(nx, ny)
                        env_color_at_tile = self.env.get_environment_color(nx, ny)

                        # 학습된 자원 선호도 점수 반영
                        learned_score = self.learned_resource_pref_scores.get(env_color_at_tile, 0.0)

                        # 먹이량과 학습된 선호도 점수를 합산하여 유효 점수 계산
                        effective_score = food_amount + (learned_score * SimulationSettings.PREY_RESOURCE_PREF_BONUS)

                        potential_target_positions.append(((nx, ny), effective_score))

                        if effective_score > max_effective_score:
                            max_effective_score = effective_score

            next_step_target = None

            if potential_target_positions:
                # 가장 높은 유효 점수를 가진 타일 중 무작위 선택
                best_options = [pos_data[0] for pos_data in potential_target_positions if pos_data[1] == max_effective_score]
                next_step_target = random.choice(best_options)
            else:
                # 주변에 유효한 타일이 없거나 먹이/선호도 점수가 모두 낮을 경우 무작위 이동
                valid_random_moves = []
                for dx_rand, dy_rand in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                    nx_rand, ny_rand = current_x + dx_rand, current_y + dy_rand
                    if self.env.is_valid_position(nx_rand, ny_rand) and not self.env.is_barrier(nx_rand, ny_rand):
                        valid_random_moves.append((nx_rand, ny_rand))

                if valid_random_moves:
                    next_step_target = random.choice(valid_random_moves)
                else:
                    next_step_target = (current_x, current_y) # 움직일 곳 없으면 제자리

            current_x, current_y = next_step_target

        self.x, self.y = current_x, current_y
        self.env.add_individual_to_occupancy_map(self)

        # 사용된 전략에 대한 피드백 (항상 'normal'로 가정, 동적으로 선택되지 않으므로)
        # 만약 foraging_strategy가 매 턴 동적으로 선택된다면 이 부분에서 전략 성공 여부에 대한 피드백을 주어야 함.
        # 현재는 이동 결과가 좋으면 strategy_use 성공으로 간주
        # self.process_feedback('foraging_strategy_use', True, chosen_strategy)


    def get_food_intake_capacity(self):
        """
        피식자의 먹이 섭취 능력을 계산합니다.
        자신의 자원 선호도와 현재 환경의 색상이 일치할수록 먹이 섭취 효율이 증가합니다.
        """
        base_capacity = SimulationSettings.PREY_FOOD_INTAKE_BASE_CAPACITY + \
                        (self.size * SimulationSettings.PREY_FOOD_INTAKE_SIZE_BONUS)

        env_color = self.env.get_environment_color(self.x, self.y)
        preference_bonus = 0
        if self.resource_preference == env_color:
            preference_bonus = SimulationSettings.PREY_RESOURCE_PREF_BONUS

        # 학습된 자원 선호도가 높을수록 먹이 섭취 효율 추가
        learned_resource_bonus = self.learned_resource_pref_scores.get(env_color, 0.0) * (SimulationSettings.PREY_RESOURCE_PREF_BONUS / 2) # 학습된 점수도 보너스로 작용

        return base_capacity + preference_bonus + learned_resource_bonus

    def get_escape_chance(self, predator_size, predator_muscle_mass):
        # 포식자로부터 도망칠 확률을 계산합니다.
        base_escape = 0.7
        size_effect = (self.size - 1) * SimulationSettings.PREY_SIZE_EFFECT_ON_ESCAPE
        muscle_effect = (self.muscle_mass - 1) * SimulationSettings.PREY_MUSCLE_EFFECT_ON_ESCAPE
        camouflage_effect = self.get_color_match_score() * SimulationSettings.PREY_CAMOUFLAGE_EFFECT_ON_ESCAPE

        predator_size_penalty = (predator_size - 1) * SimulationSettings.PREDATOR_SIZE_PENALTY_ON_ESCAPE
        predator_muscle_penalty = (predator_muscle_mass - 1) * SimulationSettings.PREDATOR_MUSCLE_PENALTY_ON_ESCAPE

        escape_chance = base_escape + size_effect + muscle_effect + camouflage_effect \
                        - predator_size_penalty - predator_muscle_penalty

        # 'stealth' 전략 보너스 적용
        if self.foraging_strategy == 'stealth':
            escape_chance += SimulationSettings.PREY_STEALTH_ESCAPE_BONUS

        return max(0.1, min(0.9, escape_chance))

    def reproduce(self, partner, is_gene_based_sim):
        # 피식자의 번식을 처리합니다.
        if not self.is_alive or not partner.is_alive: return []

        if self.age < self.reproduction_cycle or partner.age < partner.reproduction_cycle:
            return []
        if self.energy < self.max_energy * SimulationSettings.PREY_MIN_REPRO_ENERGY_FACTOR or \
           partner.energy < partner.max_energy * SimulationSettings.PREY_MIN_REPRO_ENERGY_FACTOR:
            return []

        reproduction_chance_multiplier = 1.0

        # 번식 주기 불일치 시 성공 확률 감소
        if self.reproduction_cycle != partner.reproduction_cycle:
            reproduction_chance_multiplier *= SimulationSettings.REPRODUCTION_CYCLE_MISMATCH_PENALTY

        # 유전적 유사성 기반 번식 성공률 적용 (기존 유지)
        genetic_similarity = 1.0
        if is_gene_based_sim:
            genetic_similarity = self._calculate_genetic_similarity(partner)
            if genetic_similarity < SimulationSettings.GENETIC_SIMILARITY_THRESHOLD:
                reproduction_chance_multiplier *= SimulationSettings.GENETIC_SIMILARITY_PENALTY_FACTOR

        # === 종분화를 위한 핵심 변경 부분: 짝짓기 호환성 적용 ===
        mating_compatibility = self._calculate_mating_compatibility(partner)

        # 최소 호환성 미만이면 번식 불가능
        if mating_compatibility < SimulationSettings.MIN_REPRODUCTION_COMPATIBILITY:
            self.env.simulation_ref.turn_logs[self.env.current_turn].append(
                f"피식자 {self.id} (계통: {self.lineage}, 색상: {self.color})와 {partner.id} (계통: {partner.lineage}, 색상: {partner.color})의 번식 시도 실패: 호환성 부족 ({mating_compatibility:.2f})"
            )
            self.process_feedback('mating_success', False, { # 실패 피드백
                'partner_color': partner.color,
                'partner_lineage_type': 'Same' if self.lineage == partner.lineage else 'Different'
            })
            return []

        # 호환성 점수를 번식 성공률에 직접 반영
        reproduction_chance_multiplier *= (mating_compatibility * SimulationSettings.TRAIT_DIVERGENCE_PENALTY_RATE)
        # ======================================================

        reproduction_chance_multiplier = max(SimulationSettings.MIN_REPRODUCTION_CHANCE, reproduction_chance_multiplier)

        current_food_on_tile = self.env.get_food_amount(self.x, self.y)
        food_bonus_factor = current_food_on_tile * 1.1 / self.env.max_food_per_tile

        offspring_count_this_pair = round(((self.offspring_count_base + partner.offspring_count_base) / 2) * \
                                         (1 + food_bonus_factor * SimulationSettings.PREY_REPRO_FOOD_BONUS_FACTOR))
        offspring_count_this_pair = min(offspring_count_this_pair, 100)

        if offspring_count_this_pair <= 0:
            self.process_feedback('mating_success', False, { # 실패 피드백
                'partner_color': partner.color,
                'partner_lineage_type': 'Same' if self.lineage == partner.lineage else 'Different'
            })
            return []

        new_offsprings = []
        is_reproduction_successful = False
        if random.random() < (1.0 * reproduction_chance_multiplier):
            is_reproduction_successful = True
            self.energy -= self.max_energy * SimulationSettings.PREY_REPRO_ENERGY_COST_FACTOR
            partner.energy -= partner.max_energy * SimulationSettings.PREY_REPRO_ENERGY_COST_FACTOR

            for _ in range(offspring_count_this_pair):
                child_genotype_data, child_phenotype_data = self._create_child_properties(
                    self.genotype, partner.genotype, is_gene_based_sim
                )

                possible_spawn_coords = []
                for dx in [-1, 0, 1]:
                    for dy in [-1, 0, 1]:
                        if dx == 0 and dy == 0: continue
                        spawn_x, spawn_y = self.x + dx, self.y + dy
                        if self.env.is_valid_position(spawn_x, spawn_y) and \
                           self.env.is_position_empty(spawn_x, spawn_y) and \
                           not self.env.is_barrier(spawn_x, spawn_y):
                            possible_spawn_coords.append((spawn_x, spawn_y))

                if possible_spawn_coords:
                    spawn_x, spawn_y = random.choice(possible_spawn_coords)
                    new_prey = Prey(self.env, spawn_x, spawn_y, age=0, energy=self.max_energy/2, parent_id=self.id, birth_turn=self.env.current_turn)

                    new_prey.size = child_phenotype_data['size']
                    new_prey.muscle_mass = child_phenotype_data['muscle_mass']
                    new_prey.color = child_phenotype_data['color']
                    new_prey.reproduction_cycle = child_phenotype_data['reproduction_cycle']
                    new_prey.offspring_count_base = child_phenotype_data['offspring_count_base']
                    new_prey.lineage = child_phenotype_data['lineage']
                    new_prey.resource_preference = child_phenotype_data['resource_preference']
                    new_prey.lifespan_potential = child_phenotype_data['lifespan_potential']
                    new_prey.genotype = child_genotype_data

                    new_prey.mate_pref_color = child_phenotype_data.get('mate_pref_color', 'Any')
                    new_prey.mate_pref_lineage = child_phenotype_data.get('mate_pref_lineage', 'Any')
                    new_prey.foraging_strategy = child_phenotype_data.get('foraging_strategy', 'normal')
                    new_prey.learning_rate = child_phenotype_data.get('learning_rate', 0.5) # 학습률 계승

                    # 자손의 학습된 선호도 초기화 (유전적 선호도 반영)
                    new_prey.learned_resource_pref_scores = new_prey._initialize_learned_resource_pref()
                    new_prey.learned_mating_pref_scores = new_prey._initialize_learned_mating_pref()
                    new_prey.learned_foraging_strategy_scores = new_prey._initialize_learned_foraging_strategy()

                    new_offsprings.append(new_prey)
                    log_message_suffix = ""
                    if is_gene_based_sim:
                        log_message_suffix = f", 유사성: {genetic_similarity:.2f}, 호환성: {mating_compatibility:.2f}"
                    self.env.simulation_ref.turn_logs[self.env.current_turn].append(f"피식자 {self.id} (부모 {partner.id})가 턴 {self.env.current_turn}에 새로운 피식자 {new_prey.id}를 생성했습니다. (출생 턴: {new_prey.birth_turn}, 계통: {new_prey.lineage}, 선호색: {new_prey.mate_pref_color}, 선호계통: {new_prey.mate_pref_lineage}, 전략: {new_prey.foraging_strategy}, 학습률: {new_prey.learning_rate:.2f}{log_message_suffix})")
                else:
                    self.env.simulation_ref.turn_logs[self.env.current_turn].append(
                        f"피식자 {self.id} (부모 {partner.id})가 턴 {self.env.current_turn}에 번식했지만, 자손을 배치할 공간이 없습니다."
                    )

        # 번식 성공/실패에 대한 피드백
        self.process_feedback('mating_success', is_reproduction_successful, {
            'partner_color': partner.color,
            'partner_lineage_type': 'Same' if self.lineage == partner.lineage else 'Different' # 상대방의 계통이 나와 같은지 다른지
        })

        return new_offsprings


# Predator 클래스 정의 (포식자)
class Predator(Individual):
    def __init__(self, env, x, y, age=0, energy=SimulationSettings.MAX_ENERGY_PREDATOR / 2, parent_id=None, birth_turn=None):
        super().__init__(env, x, y, age, energy, parent_id, birth_turn=birth_turn)
        self.max_energy = SimulationSettings.MAX_ENERGY_PREDATOR
        self.base_energy_consumption = SimulationSettings.BASE_ENERGY_CONSUMPTION_PREDATOR
        self.max_age = SimulationSettings.MAX_AGE_PREDATOR
        self.carcass_food_value = SimulationSettings.CARCASS_FOOD_VALUE_PREDATOR
        self.hunting_range = SimulationSettings.PREDATOR_HUNTING_RANGE

        self.base_movement_speed = SimulationSettings.BASE_MOVEMENT_SPEED_PREDATOR

    def live_a_turn(self, all_predators):
        # 한 턴 동안 포식자의 행동 (나이 증가, 에너지 소비, 이동)을 처리합니다.
        if not self.is_alive: return

        if self.age_and_check_death(): return

        self.energy -= self.get_energy_consumption()
        if self.energy <= 0:
            self.die("굶주림")
            return

        # 번식 우선순위 로직 (매력도 기반으로 파트너 찾기)
        if self.age >= self.reproduction_cycle - 1 and self.energy >= self.max_energy * SimulationSettings.PREDATOR_MIN_REPRO_ENERGY_FACTOR:
            best_partner_pos = None
            max_attractiveness = -1.0

            for other_predator_id in all_predators:
                other_predator = all_predators[other_predator_id]
                if other_predator.is_alive and other_predator.id != self.id and \
                   other_predator.age >= other_predator.reproduction_cycle - 1:

                    attractiveness = self.get_mating_attractiveness(other_predator)
                    if attractiveness > max_attractiveness:
                        max_attractiveness = attractiveness
                        best_partner_pos = (other_predator.x, other_predator.y)

            if best_partner_pos and max_attractiveness > 0:
                self._move_towards_target(best_partner_pos[0], best_partner_pos[1])
            else:
                self._perform_hunting_move() # 파트너 없거나 매력 없으면 사냥
        else:
            self._perform_hunting_move()

    def _perform_hunting_move(self):
        # 포식자 개체를 환경 내에서 이동시킵니다.
        if not self.is_alive: return

        self.env.remove_individual_from_occupancy_map(self)

        current_x, current_y = self.x, self.y
        speed = self.get_effective_movement_speed()

        chosen_strategy = self.hunting_strategy # 현재는 고정된 전략 사용

        target_prey_pos = None
        min_dist_to_prey = float('inf')

        # 사냥 범위 내에서 가장 가까운 피식자를 찾습니다.
        for prey_id in self.env.simulation_ref.prey:
            prey_obj = self.env.simulation_ref.prey[prey_id]
            if prey_obj.is_alive:
                dist = abs(self.x - prey_obj.x) + abs(self.y - prey_obj.y)
                # 매복 전략은 최소 사냥 범위 내에서만 반응 (ambush)
                if chosen_strategy == 'ambush' and dist <= SimulationSettings.PREDATOR_AMBUSH_MIN_HUNTING_RANGE:
                    if dist < min_dist_to_prey:
                        min_dist_to_prey = dist
                        target_prey_pos = (prey_obj.x, prey_obj.y)
                # 추적 전략은 넓은 사냥 범위 내에서 반응 (pursuit)
                elif chosen_strategy == 'pursuit' and dist <= self.hunting_range:
                    if dist < min_dist_to_prey:
                        min_dist_to_prey = dist
                        target_prey_pos = (prey_obj.x, prey_obj.y)

        # 전략에 따른 이동 로직
        if target_prey_pos:
            # 목표를 향해 이동
            for _ in range(speed):
                dx, dy = 0, 0
                if target_prey_pos[0] > current_x: dx = 1
                elif target_prey_pos[0] < current_x: dx = -1

                if target_prey_pos[1] > current_y: dy = 1
                elif target_prey_pos[1] < current_y: dy = -1

                if dx == 0 and dy == 0: break

                next_x, next_y = current_x + dx, current_y + dy
                if self.env.is_valid_position(next_x, next_y) and not self.env.is_barrier(next_x, next_y):
                    current_x, current_y = next_x, next_y
                else:
                    valid_random_moves = []
                    for rdx, rdy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                        rand_x, rand_y = current_x + rdx, current_y + rdy
                        if self.env.is_valid_position(rand_x, rand_y) and not self.env.is_barrier(rand_x, rand_y): # 수정된 ny_rand -> rand_y
                            valid_random_moves.append((rand_x, rand_y))

                    if valid_random_moves:
                        current_x, current_y = random.choice(valid_random_moves)
                    else:
                        break
        else: # 피식자가 없으면 무작위 이동
            for _ in range(speed):
                dx, dy = random.choice([(0, 1), (0, -1), (1, 0), (-1, 0)])
                new_x, new_y = current_x + dx, current_y + dy
                if self.env.is_valid_position(new_x, new_y) and not self.env.is_barrier(new_x, new_y):
                    current_x, current_y = new_x, new_y
                else:
                    break

        self.x, self.y = current_x, current_y
        self.env.add_individual_to_occupancy_map(self)


    def get_hunt_success_chance(self, prey_size, prey_muscle_mass, prey_color, prey_pos_color, dist_to_prey):
        # 사냥 성공 확률을 계산합니다.
        base_success_chance = 0.6

        pred_size_effect = (self.size - 1) * SimulationSettings.PRED_SIZE_EFFECT_ON_HUNT
        pred_muscle_effect = (self.muscle_mass - 1) * SimulationSettings.PRED_MUSCLE_EFFECT_ON_HUNT
        pred_camouflage_effect = self.get_color_match_score() * SimulationSettings.PRED_CAMOUFLAGE_EFFECT_ON_HUNT

        prey_size_penalty = (prey_size - 1) * SimulationSettings.PREY_SIZE_PENALTY_ON_HUNT
        prey_muscle_penalty = (prey_muscle_mass - 1) * SimulationSettings.PREY_MUSCLE_PENALTY_ON_HUNT
        prey_camouflage_penalty = self.get_color_match_score_prey(prey_color, prey_pos_color) * SimulationSettings.PREY_CAMOUFLAGE_PENALTY_ON_HUNT

        success_chance = base_success_chance + pred_size_effect + pred_muscle_effect + pred_camouflage_effect \
                         - prey_size_penalty - prey_muscle_penalty - prey_camouflage_penalty

        # 'ambush' 전략 보너스 적용
        if self.hunting_strategy == 'ambush' and dist_to_prey <= SimulationSettings.PREDATOR_AMBUSH_MIN_HUNTING_RANGE:
            success_chance += SimulationSettings.PREDATOR_AMBUSH_HUNT_SUCCESS_BONUS

        return max(0.1, min(0.9, success_chance))

    def get_color_match_score_prey(self, prey_color, prey_pos_color):
        # 피식자의 위장 점수를 계산합니다.
        if prey_color == prey_pos_color:
            return 1.0
        elif (prey_color == 'Purple' and (prey_pos_color == 'Red' or prey_pos_color == 'Blue')) or \
             ((prey_color == 'Red' or prey_pos_color == 'Blue') and prey_pos_color == 'Purple'): # 여기 논리 오류 수정
            return 0.5
        else:
            return 0.0

    def reproduce(self, partner, is_gene_based_sim):
        # 포식자의 번식을 처리합니다.
        if not self.is_alive or not partner.is_alive: return []

        if self.age < self.reproduction_cycle or partner.age < partner.reproduction_cycle:
            return []
        if self.energy < self.max_energy * SimulationSettings.PREDATOR_MIN_REPRO_ENERGY_FACTOR or \
           partner.energy < partner.max_energy * SimulationSettings.PREDATOR_MIN_REPRO_ENERGY_FACTOR:
            return []

        reproduction_chance_multiplier = 1.0

        if self.reproduction_cycle != partner.reproduction_cycle:
            reproduction_chance_multiplier *= SimulationSettings.REPRODUCTION_CYCLE_MISMATCH_PENALTY

        # 유전적 유사성 기반 번식 성공률 적용 (기존 유지)
        genetic_similarity = 1.0
        if is_gene_based_sim:
            genetic_similarity = self._calculate_genetic_similarity(partner)
            if genetic_similarity < SimulationSettings.GENETIC_SIMILARITY_THRESHOLD:
                reproduction_chance_multiplier *= SimulationSettings.GENETIC_SIMILARITY_PENALTY_FACTOR

        # === 종분화를 위한 핵심 변경 부분: 짝짓기 호환성 적용 ===
        mating_compatibility = self._calculate_mating_compatibility(partner)

        # 최소 호환성 미만이면 번식 불가능
        if mating_compatibility < SimulationSettings.MIN_REPRODUCTION_COMPATIBILITY:
            self.env.simulation_ref.turn_logs[self.env.current_turn].append(
                f"포식자 {self.id} (계통: {self.lineage}, 색상: {self.color})와 {partner.id} (계통: {partner.lineage}, 색상: {partner.color})의 번식 시도 실패: 호환성 부족 ({mating_compatibility:.2f})"
            )
            self.process_feedback('mating_success', False, { # 실패 피드백
                'partner_color': partner.color,
                'partner_lineage_type': 'Same' if self.lineage == partner.lineage else 'Different'
            })
            return []

        # 호환성 점수를 번식 성공률에 직접 반영
        reproduction_chance_multiplier *= (mating_compatibility * SimulationSettings.TRAIT_DIVERGENCE_PENALTY_RATE)
        # ======================================================

        reproduction_chance_multiplier = max(SimulationSettings.MIN_REPRODUCTION_CHANCE, reproduction_chance_multiplier)

        avg_energy = (self.energy + partner.energy) / 2
        energy_bonus_factor = avg_energy / self.max_energy

        offspring_count_this_pair = round(((self.offspring_count_base + partner.offspring_count_base) / 2) * \
                                         (1 + energy_bonus_factor * SimulationSettings.PREDATOR_REPRO_ENERGY_BONUS_FACTOR))
        offspring_count_this_pair = min(offspring_count_this_pair, 3)

        if offspring_count_this_pair <= 0:
            self.process_feedback('mating_success', False, { # 실패 피드백
                'partner_color': partner.color,
                'partner_lineage_type': 'Same' if self.lineage == partner.lineage else 'Different'
            })
            return []

        new_offsprings = []
        is_reproduction_successful = False
        if random.random() < (0.7 * reproduction_chance_multiplier):
            is_reproduction_successful = True
            self.energy -= self.max_energy * SimulationSettings.PREDATOR_REPRO_ENERGY_COST_FACTOR
            partner.energy -= partner.max_energy * SimulationSettings.PREDATOR_REPRO_ENERGY_COST_FACTOR

            for _ in range(offspring_count_this_pair):
                child_genotype_data, child_phenotype_data = self._create_child_properties(
                    self.genotype, partner.genotype, is_gene_based_sim
                )

                possible_spawn_coords = []
                for dx in [-1, 0, 1]:
                    for dy in [-1, 0, 1]:
                        if dx == 0 and dy == 0: continue
                        spawn_x, spawn_y = self.x + dx, self.y + dy
                        if self.env.is_valid_position(spawn_x, spawn_y) and \
                           self.env.is_position_empty(spawn_x, spawn_y) and \
                           not self.env.is_barrier(spawn_x, spawn_y):
                            possible_spawn_coords.append((spawn_x, spawn_y))

                if possible_spawn_coords:
                    spawn_x, spawn_y = random.choice(possible_spawn_coords)
                    new_predator = Predator(self.env, spawn_x, spawn_y, age=0, energy=self.max_energy/2, parent_id=self.id, birth_turn=self.env.current_turn)

                    new_predator.size = child_phenotype_data['size']
                    new_predator.muscle_mass = child_phenotype_data['muscle_mass']
                    new_predator.color = child_phenotype_data['color']
                    new_predator.reproduction_cycle = child_phenotype_data['reproduction_cycle']
                    new_predator.offspring_count_base = child_phenotype_data['offspring_count_base']
                    new_predator.lineage = child_phenotype_data['lineage']
                    new_predator.resource_preference = child_phenotype_data['resource_preference']
                    new_predator.lifespan_potential = child_phenotype_data['lifespan_potential']
                    new_predator.genotype = child_genotype_data

                    new_predator.mate_pref_color = child_phenotype_data.get('mate_pref_color', 'Any')
                    new_predator.mate_pref_lineage = child_phenotype_data.get('mate_pref_lineage', 'Any')
                    new_predator.hunting_strategy = child_phenotype_data.get('hunting_strategy', 'pursuit')
                    new_predator.learning_rate = child_phenotype_data.get('learning_rate', 0.5) # 학습률 계승

                    # 자손의 학습된 선호도 초기화 (유전적 선호도 반영)
                    new_predator.learned_resource_pref_scores = new_predator._initialize_learned_resource_pref()
                    new_predator.learned_mating_pref_scores = new_predator._initialize_learned_mating_pref()
                    new_predator.learned_hunting_strategy_scores = new_predator._initialize_learned_hunting_strategy()

                    new_offsprings.append(new_predator)
                    log_message_suffix = ""
                    if is_gene_based_sim:
                        log_message_suffix = f", 유사성: {genetic_similarity:.2f}, 호환성: {mating_compatibility:.2f}"
                    self.env.simulation_ref.turn_logs[self.env.current_turn].append(f"포식자 {self.id} (부모 {partner.id})가 턴 {self.env.current_turn}에 새로운 포식자 {new_predator.id}를 생성했습니다. (출생 턴: {new_predator.birth_turn}, 계통: {new_predator.lineage}, 선호색: {new_predator.mate_pref_color}, 선호계통: {new_predator.mate_pref_lineage}, 전략: {new_predator.hunting_strategy}, 학습률: {new_predator.learning_rate:.2f}{log_message_suffix})")
                else:
                    self.env.simulation_ref.turn_logs[self.env.current_turn].append(
                        f"포식자 {self.id} (부모 {partner.id})가 턴 {self.env.current_turn}에 번식했지만, 자손을 배치할 공간이 없습니다."
                    )

        # 번식 성공/실패에 대한 피드백
        self.process_feedback('mating_success', is_reproduction_successful, {
            'partner_color': partner.color,
            'partner_lineage_type': 'Same' if self.lineage == partner.lineage else 'Different'
        })
        return new_offsprings


# Simulation 클래스 정의 (전체 시뮬레이션 제어)
class Simulation:
    def __init__(self, width, height, max_food_per_tile, food_regen_rate,
                         initial_prey_count, initial_predator_count,
                         max_turns, is_gene_based_sim, seed=None ):

        # 시드 설정 및 저장
        if seed is not None:
            random.seed(seed)
            self._used_seed = seed
        else:
            current_time_seed = int(time.time() * 1000)
            random.seed(current_time_seed)
            self._used_seed = current_time_seed

        self.env = Environment(width, height, max_food_per_tile, food_regen_rate)
        self.env.simulation_ref = self
        self.env.current_turn = 0
        self.prey = {}
        self.predators = {}
        self.max_turns = max_turns
        self.is_gene_based_sim = is_gene_based_sim

        self.turn = 0
        self.simulation_data = collections.defaultdict(dict)
        self.turn_logs = collections.defaultdict(list)

        self._initialize_population(initial_prey_count, initial_predator_count)

        self.turn_logs[0].append(f"시뮬레이션 시작 시드: {self._used_seed}")

    def _initialize_population(self, initial_prey_count, initial_predator_count):
        Individual._next_id = 0

        for _ in range(initial_prey_count):
            x, y = self._get_random_empty_position()
            if x is None: continue

            new_prey = Prey(self.env, x, y, birth_turn=0)

            if not self.is_gene_based_sim:
                # 무작위 시뮬레이션일 경우, Individual.__init__에서 생성된 초기값을 무작위로 다시 할당
                new_prey.size = new_prey._generate_random_trait_value('size')
                new_prey.muscle_mass = new_prey._generate_random_trait_value('muscle_mass')
                new_prey.color = new_prey._generate_random_trait_value('color')
                new_prey.reproduction_cycle = new_prey._generate_random_trait_value('reproduction_cycle', 'Prey')
                new_prey.offspring_count_base = new_prey._generate_random_trait_value('offspring_count_base')
                new_prey.lineage = new_prey._generate_random_trait_value('lineage')
                new_prey.resource_preference = new_prey._generate_random_trait_value('resource_preference')
                new_prey.lifespan_potential = new_prey._generate_random_trait_value('lifespan_potential')

                # 새로운 형질 무작위 할당
                new_prey.mate_pref_color = new_prey._generate_random_trait_value('mate_pref_color')
                new_prey.mate_pref_lineage = new_prey._generate_random_trait_value('mate_pref_lineage')
                new_prey.foraging_strategy = new_prey._generate_random_trait_value('foraging_strategy')
                new_prey.learning_rate = new_prey._generate_random_trait_value('learning_rate') # 학습률 초기화

                new_prey.genotype = new_prey._generate_genotype_from_phenotype()

                # 학습된 선호도 초기화 (무작위 시뮬레이션에서도 초기화)
                new_prey.learned_resource_pref_scores = new_prey._initialize_learned_resource_pref()
                new_prey.learned_mating_pref_scores = new_prey._initialize_learned_mating_pref()
                new_prey.learned_foraging_strategy_scores = new_prey._initialize_learned_foraging_strategy()


            self.prey[new_prey.id] = new_prey
            self.turn_logs[0].append(f"초기 피식자 {new_prey.id}가 턴 0에 생성되었습니다. (Pos:({new_prey.x},{new_prey.y}), 계통: {new_prey.lineage}, 선호색: {new_prey.mate_pref_color}, 선호계통: {new_prey.mate_pref_lineage}, 전략: {new_prey.foraging_strategy}, 학습률: {new_prey.learning_rate:.2f})")


        for _ in range(initial_predator_count):
            x, y = self._get_random_empty_position()
            if x is None: continue

            new_predator = Predator(self.env, x, y, birth_turn=0)

            if not self.is_gene_based_sim:
                new_predator.size = new_predator._generate_random_trait_value('size')
                new_predator.muscle_mass = new_predator._generate_random_trait_value('muscle_mass')
                new_predator.color = new_predator._generate_random_trait_value('color')
                new_predator.reproduction_cycle = new_predator._generate_random_trait_value('reproduction_cycle', 'Predator')
                new_predator.offspring_count_base = new_predator._generate_random_trait_value('offspring_count_base')
                new_predator.lineage = new_predator._generate_random_trait_value('lineage')
                new_predator.resource_preference = new_predator._generate_random_trait_value('resource_preference')
                new_predator.lifespan_potential = new_predator._generate_random_trait_value('lifespan_potential')

                # 새로운 형질 무작위 할당
                new_predator.mate_pref_color = new_predator._generate_random_trait_value('mate_pref_color')
                new_predator.mate_pref_lineage = new_predator._generate_random_trait_value('mate_pref_lineage')
                new_predator.hunting_strategy = new_predator._generate_random_trait_value('hunting_strategy')
                new_predator.learning_rate = new_predator._generate_random_trait_value('learning_rate') # 학습률 초기화

                new_predator.genotype = new_predator._generate_genotype_from_phenotype()

                # 학습된 선호도 초기화
                new_predator.learned_resource_pref_scores = new_predator._initialize_learned_resource_pref()
                new_predator.learned_mating_pref_scores = new_predator._initialize_learned_mating_pref()
                new_predator.learned_hunting_strategy_scores = new_predator._initialize_learned_hunting_strategy()


            self.predators[new_predator.id] = new_predator
            self.turn_logs[0].append(f"초기 포식자 {new_predator.id}가 턴 0에 생성되었습니다. (Pos:({new_predator.x},{new_predator.y}), 계통: {new_predator.lineage}, 선호색: {new_predator.mate_pref_color}, 선호계통: {new_predator.mate_pref_lineage}, 전략: {new_predator.hunting_strategy}, 학습률: {new_predator.learning_rate:.2f})")

    def _get_random_empty_position(self):
        # 환경 그리드 내에서 무작위의 빈 위치 (아무 개체도 없는)를 찾습니다.
        possible_positions = []
        for x in range(self.env.width):
            for y in range(self.env.height):
                if self.env.is_position_empty(x, y) and not self.env.is_barrier(x, y):
                    possible_positions.append((x, y))

        if not possible_positions:
            return None, None
        return random.choice(possible_positions)

    def run_turn(self):
        # 시뮬레이션의 한 턴을 실행합니다.
        self.turn += 1
        self.env.current_turn = self.turn

        self.env.update_food()
        self.env.update_barriers(self.turn)

        dead_prey_ids = []
        dead_predator_ids = []

        new_prey_offsprings = []
        new_predator_offsprings = []

        # 1. 피식자 행동
        for prey_id in list(self.prey.keys()):
            prey = self.prey.get(prey_id)
            if not prey or not prey.is_alive: continue

            prey.live_a_turn(self.prey) # 피식자 턴 실행
            if not prey.is_alive:
                dead_prey_ids.append(prey_id)
                continue

            # 번식 시도 (live_a_turn에서 파트너에게 이동했으므로, 이제 같은 칸에 있는지 확인)
            found_partner_at_same_pos = None
            for other_id in self.env.occupancy_map[(prey.x, prey.y)]:
                other_prey = self.prey.get(other_id)
                if other_prey and other_prey.is_alive and other_prey.id != prey.id:
                    found_partner_at_same_pos = other_prey
                    break

            if found_partner_at_same_pos: # 같은 칸에 파트너가 있다면 번식 시도
                offsprings = prey.reproduce(found_partner_at_same_pos, self.is_gene_based_sim)
                for child in offsprings:
                    new_prey_offsprings.append(child)

        # 2. 포식자 행동
        for predator_id in list(self.predators.keys()):
            predator = self.predators.get(predator_id)
            if not predator or not predator.is_alive: continue

            predator.live_a_turn(self.predators) # 포식자 턴 실행
            if not predator.is_alive:
                dead_predator_ids.append(predator_id)
                continue

            # 사냥 성공 여부 판단 및 에너지 획득 (포식자가 피식자와 같은 칸에 있는지 확인)
            hunted_prey_id = None
            prey_at_current_pos = [self.prey.get(pid) for pid in self.env.occupancy_map[(predator.x, predator.y)] if self.prey.get(pid) and self.prey.get(pid).is_alive]

            if prey_at_current_pos:
                target_prey = random.choice(prey_at_current_pos)
                dist_to_prey_current = 0 # 현재 위치이므로 거리 0

                env_color_at_prey = self.env.get_environment_color(target_prey.x, target_prey.y)
                success_chance = predator.get_hunt_success_chance(
                    target_prey.size, target_prey.muscle_mass, target_prey.color, env_color_at_prey, dist_to_prey_current
                )

                hunt_successful = False
                if random.random() < success_chance:
                    hunt_successful = True
                    predator.energy = min(predator.max_energy, predator.energy + target_prey.max_energy)
                    target_prey.die("사냥")
                    hunted_prey_id = target_prey.id

                # 사냥 성공/실패에 대한 피드백 처리
                predator.process_feedback('hunting_success', hunt_successful, predator.hunting_strategy) # 사용한 전략에 대한 피드백

            if hunted_prey_id is not None:
                dead_prey_ids.append(hunted_prey_id)

            # 번식 시도 (live_a_turn에서 파트너에게 이동했으므로, 이제 같은 칸에 있는지 확인)
            found_partner_at_same_pos = None
            for other_id in self.env.occupancy_map[(predator.x, predator.y)]:
                other_predator = self.predators.get(other_id)
                if other_predator and other_predator.is_alive and other_predator.id != predator.id:
                    found_partner_at_same_pos = other_predator
                    break

            if found_partner_at_same_pos: # 같은 칸에 파트너가 있다면 번식 시도
                offsprings = predator.reproduce(found_partner_at_same_pos, self.is_gene_based_sim)
                for child in offsprings:
                    new_predator_offsprings.append(child)

        # 3. 죽은 개체 제거
        for prey_id in list(set(dead_prey_ids)):
            if prey_id in self.prey:
                del self.prey[prey_id]

        for predator_id in dead_predator_ids:
            if predator_id in self.predators:
                del self.predators[predator_id]

        # 4. 새로운 개체 추가
        for new_prey in new_prey_offsprings:
            self.prey[new_prey.id] = new_prey
        for new_predator in new_predator_offsprings:
            self.predators[new_predator.id] = new_predator

        self._collect_data()

        if not self.prey and not self.predators:
            self.turn_logs[self.turn].append(f"턴 {self.turn}에 모든 개체가 멸종했습니다. 시뮬레이션 종료.")
            return False
        elif not self.prey:
            self.turn_logs[self.turn].append(f"턴 {self.turn}에 모든 피식자가 멸종했습니다. 시뮬레이션 종료.")
            return False
        elif not self.predators:
            self.turn_logs[self.turn].append(f"턴 {self.turn}에 모든 포식자가 멸종했습니다. 시뮬레이션 종료.")
            return False
        elif self.turn >= self.max_turns:
            self.turn_logs[self.turn].append(f"최대 턴 ({self.max_turns})에 도달하여 시뮬레이션을 종료합니다.")
            return False
        return True

    def _collect_data(self):
        # 현재 턴의 시뮬레이션 데이터를 수집하여 저장합니다.
        self.simulation_data[self.turn]['prey_count'] = len(self.prey)
        self.simulation_data[self.turn]['predator_count'] = len(self.predators)

        # 피식자 형질 데이터 수집
        prey_sizes = [p.size for p in self.prey.values()]
        prey_muscles = [p.muscle_mass for p in self.prey.values()]
        prey_colors = [p.color for p in self.prey.values()]
        prey_repro_cycles = [p.reproduction_cycle for p in self.prey.values()]
        prey_offspring_bases = [p.offspring_count_base for p in self.prey.values()]
        prey_resource_prefs = [p.resource_preference for p in self.prey.values()]
        prey_lifespan_pots = [p.lifespan_potential for p in self.prey.values()]
        prey_lineages = [p.lineage for p in self.prey.values()]
        # 새로운 형질 수집
        prey_mate_pref_colors = [p.mate_pref_color for p in self.prey.values()]
        prey_mate_pref_lineages = [p.mate_pref_lineage for p in self.prey.values()]
        prey_foraging_strategies = [p.foraging_strategy for p in self.prey.values()]
        prey_learning_rates = [p.learning_rate for p in self.prey.values()] # 학습률 수집

        # 포식자 형질 데이터 수집
        predator_sizes = [p.size for p in self.predators.values()]
        predator_muscles = [p.muscle_mass for p in self.predators.values()]
        predator_colors = [p.color for p in self.predators.values()]
        predator_repro_cycles = [p.reproduction_cycle for p in self.predators.values()]
        predator_offspring_bases = [p.offspring_count_base for p in self.predators.values()]
        predator_resource_prefs = [p.resource_preference for p in self.predators.values()]
        predator_lifespan_pots = [p.lifespan_potential for p in self.predators.values()]
        predator_lineages = [p.lineage for p in self.predators.values()]
        # 새로운 형질 수집
        predator_mate_pref_colors = [p.mate_pref_color for p in self.predators.values()]
        predator_mate_pref_lineages = [p.mate_pref_lineage for p in self.predators.values()]
        predator_hunting_strategies = [p.hunting_strategy for p in self.predators.values()]
        predator_learning_rates = [p.learning_rate for p in self.predators.values()] # 학습률 수집

        self.simulation_data[self.turn]['prey_traits'] = {
            'size_avg': sum(prey_sizes) / len(prey_sizes) if prey_sizes else 0,
            'muscle_avg': sum(prey_muscles) / len(prey_muscles) if prey_muscles else 0,
            'color_dist': collections.Counter(prey_colors),
            'repro_cycle_avg': sum(prey_repro_cycles) / len(prey_repro_cycles) if prey_repro_cycles else 0,
            'offspring_base_avg': sum(prey_offspring_bases) / len(prey_offspring_bases) if prey_offspring_bases else 0,
            'resource_pref_dist': collections.Counter(prey_resource_prefs),
            'lifespan_pot_avg': sum(prey_lifespan_pots) / len(prey_lifespan_pots) if prey_lifespan_pots else 0,
            'lineage_dist': collections.Counter(prey_lineages),
            'mate_pref_color_dist': collections.Counter(prey_mate_pref_colors),
            'mate_pref_lineage_dist': collections.Counter(prey_mate_pref_lineages),
            'foraging_strategy_dist': collections.Counter(prey_foraging_strategies),
            'learning_rate_avg': sum(prey_learning_rates) / len(prey_learning_rates) if prey_learning_rates else 0, # 학습률 평균
            'learning_rate_dist': collections.Counter(round(rate * 10) / 10 for rate in prey_learning_rates) if prey_learning_rates else collections.Counter(), # 학습률 분포 (소수점 첫째 자리 반올림)
            'ids': list(self.prey.keys())
        }
        self.simulation_data[self.turn]['predator_traits'] = {
            'size_avg': sum(predator_sizes) / len(predator_sizes) if predator_sizes else 0,
            'muscle_avg': sum(predator_muscles) / len(predator_muscles) if predator_muscles else 0,
            'color_dist': collections.Counter(predator_colors),
            'repro_cycle_avg': sum(predator_repro_cycles) / len(predator_repro_cycles) if predator_repro_cycles else 0,
            'offspring_base_avg': sum(predator_offspring_bases) / len(predator_offspring_bases) if predator_offspring_bases else 0,
            'resource_pref_dist': collections.Counter(predator_resource_prefs),
            'lifespan_pot_avg': sum(predator_lifespan_pots) / len(predator_lifespan_pots) if predator_lifespan_pots else 0,
            'lineage_dist': collections.Counter(predator_lineages),
            'mate_pref_color_dist': collections.Counter(predator_mate_pref_colors),
            'mate_pref_lineage_dist': collections.Counter(predator_mate_pref_lineages),
            'hunting_strategy_dist': collections.Counter(predator_hunting_strategies),
            'learning_rate_avg': sum(predator_learning_rates) / len(predator_learning_rates) if predator_learning_rates else 0,
            'learning_rate_dist': collections.Counter(round(rate * 10) / 10 for rate in predator_learning_rates) if predator_learning_rates else collections.Counter(),
            'ids': list(self.predators.keys())
        }

    def get_simulation_results(self):
        # 시뮬레이션의 최종 결과를 반환합니다.
        return {
            'data': self.simulation_data,
            'final_turn': self.turn,
            'prey_alive': len(self.prey) > 0,
            'predator_alive': len(self.predators) > 0,
            'turn_logs': self.turn_logs
        }

    def save_turn_logs(self, filepath):
        # 턴별 로그를 파일로 저장합니다.
        with open(filepath, 'w', encoding='utf-8') as f:
            for turn in sorted(self.turn_logs.keys()):
                f.write(f"--- 턴 {turn} ---\n")
                for log_entry in self.turn_logs[turn]:
                    f.write(log_entry + "\n")
                f.write("\n")


# 여러 시뮬레이션을 실행하는 함수
def run_multiple_simulations(num_simulations, sim_type, sim_params, base_seed=None):
    all_sim_results = []
    print(f"\n--- {sim_type} 시뮬레이션 ({num_simulations}회 반복) 시작 ---")
    for i in range(num_simulations):
        current_seed = base_seed + i if base_seed is not None else None

        print(f"  > 시뮬레이션 {i+1}/{num_simulations} 실행 중 (시드: {'고정됨' if base_seed is not None else '랜덤'})...")

        sim = Simulation(
            width=sim_params['width'],
            height=sim_params['height'],
            max_food_per_tile=sim_params['max_food_per_tile'],
            food_regen_rate=sim_params['food_regen_rate'],
            initial_prey_count=sim_params['initial_prey_count'],
            initial_predator_count=sim_params['initial_predator_count'],
            max_turns=sim_params['max_turns'],
            is_gene_based_sim=(sim_type == "유전자 기반"),
            seed=current_seed
        )
        while sim.run_turn():
            pass

        all_sim_results.append(sim.get_simulation_results())
        print(f"  > 시뮬레이션 {i+1} 종료 (총 턴: {sim.turn}, 피식자: {len(sim.prey)}, 포식자: {len(sim.predators)})")
    return all_sim_results

def analyze_and_plot_results(sim_results_list, sim_type_label, output_dir, base_seed_used=None):
    print(f"\n--- {sim_type_label} 시뮬레이션 결과 분석 및 시각화 ---")

    # 결과 보고서 파일 초기화
    report_file_path = os.path.join(output_dir, f"{sim_type_label}_simulation_report.txt")
    with open(report_file_path, 'w', encoding='utf-8') as f:
        f.write(f"### {sim_type_label} 시뮬레이션 결과 보고서 ###\n\n")
        f.write(f"실행 횟수: {len(sim_results_list)}회\n")
        if base_seed_used is not None:
            f.write(f"사용된 기본 시드: {base_seed_used}\n")
        else:
            f.write(f"사용된 기본 시드: 랜덤 (각 시뮬레이션마다 고유)\n")
        f.write(f"시작 시간: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

    max_total_turns = max(results['final_turn'] for results in sim_results_list) if sim_results_list else 0

    # 1. 개체군 크기 변화 (개별 시뮬레이션별)
    plt.figure(figsize=(12, 6))
    for i, results in enumerate(sim_results_list):
        turns = sorted(results['data'].keys())
        prey_counts = [results['data'][t]['prey_count'] for t in turns]
        predator_counts = [results['data'][t]['predator_count'] for t in turns]
        plt.plot(turns, prey_counts, label=f'피식자 시뮬레이션 {i+1}회', alpha=0.5, linestyle='--')
        plt.plot(turns, predator_counts, label=f'포식자 시뮬레이션 {i+1}회', alpha=0.5)

    plt.title(f'{sim_type_label} 시나리오 - 개별 시뮬레이션별 개체군 크기 변화')
    plt.xlabel('턴')
    plt.ylabel('개체군')
    plt.legend()
    plt.grid(True)
    plot_filepath = os.path.join(output_dir, f'{sim_type_label}_individual_population_change.png')
    plt.savefig(plot_filepath)
    plt.close()
    with open(report_file_path, 'a', encoding='utf-8') as f:
        f.write(f"** 개별 시뮬레이션별 개체군 크기 변화 그래프: {os.path.basename(plot_filepath)}\n\n")


    # 1. 개체군 크기 변화 (모든 시뮬레이션의 평균)
    avg_prey_counts = np.zeros(max_total_turns + 1)
    avg_predator_counts = np.zeros(max_total_turns + 1)
    num_data_points_prey = np.zeros(max_total_turns + 1)
    num_data_points_predator = np.zeros(max_total_turns + 1)

    for results in sim_results_list:
        for t, data in results['data'].items():
            if t <= max_total_turns:
                avg_prey_counts[t] += data['prey_count']
                avg_predator_counts[t] += data['predator_count']
                if data['prey_count'] > 0 or t == 0: num_data_points_prey[t] += 1
                if data['predator_count'] > 0 or t == 0: num_data_points_predator[t] += 1

    avg_prey_counts = np.divide(avg_prey_counts, num_data_points_prey, where=num_data_points_prey!=0)
    avg_predator_counts = np.divide(avg_predator_counts, num_data_points_predator, where=num_data_points_predator!=0)

    plt.figure(figsize=(12, 6))
    plt.plot(range(max_total_turns + 1), avg_prey_counts, label='평균 피식자 개체군', color='green', linewidth=2)
    plt.plot(range(max_total_turns + 1), avg_predator_counts, label='평균 포식자 개체군', color='red', linewidth=2)
    plt.title(f'{sim_type_label} 시나리오 - 모든 시뮬레이션의 평균 개체군 크기 변화')
    plt.xlabel('턴')
    plt.ylabel('개체군')
    plt.legend()
    plt.grid(True)
    plot_filepath = os.path.join(output_dir, f'{sim_type_label}_average_population_change.png')
    plt.savefig(plot_filepath)
    plt.close()
    with open(report_file_path, 'a', encoding='utf-8') as f:
        f.write(f"** 모든 시뮬레이션의 평균 개체군 크기 변화 그래프: {os.path.basename(plot_filepath)}\n\n")


    # 2. 형질 변화 추이 (평균 및 개별 시뮬레이션)
    trait_names = ['size_avg', 'muscle_avg', 'repro_cycle_avg', 'offspring_base_avg', 'lifespan_pot_avg', 'learning_rate_avg'] # 학습률 평균 추가
    color_names = ['Red', 'Blue', 'Purple']
    resource_pref_names = ['Red', 'Blue', 'Purple']

    # 새로운 형질 분포를 위한 이름 목록
    mate_pref_color_types = SimulationSettings.MATE_PREF_COLOR_TYPES
    mate_pref_lineage_types = SimulationSettings.MATE_PREF_LINEAGE_TYPES
    foraging_strategy_names = SimulationSettings.FORAGING_STRATEGIES
    hunting_strategy_names = SimulationSettings.HUNTING_STRATEGIES

    for trait_name in trait_names:
        # 개별 시뮬레이션별 형질 변화
        plt.figure(figsize=(12, 6))
        for i, results in enumerate(sim_results_list):
            turns = sorted(results['data'].keys())
            prey_trait_values = [results['data'][t]['prey_traits'].get(trait_name, 0) for t in turns if results['data'][t].get('prey_count', 0) > 0]
            predator_trait_values = [results['data'][t]['predator_traits'].get(trait_name, 0) for t in turns if results['data'][t].get('predator_count', 0) > 0]

            prey_turns = [t for t in turns if results['data'][t].get('prey_count', 0) > 0]
            predator_turns = [t for t in turns if results['data'][t].get('predator_count', 0) > 0]

            if prey_turns:
                plt.plot(prey_turns, prey_trait_values, label=f'피식자 시뮬레이션 {i+1}회', alpha=0.5, linestyle='--')
            if predator_turns:
                plt.plot(predator_turns, predator_trait_values, label=f'포식자 시뮬레이션 {i+1}회', alpha=0.5)

        plt.title(f'{sim_type_label} 시나리오 - 개별 시뮬레이션별 {trait_name.replace("_avg", "")} 평균 변화')
        plt.xlabel('턴')
        plt.ylabel(f'{trait_name.replace("_avg", "")} 평균')
        plt.legend()
        plt.grid(True)
        plot_filepath = os.path.join(output_dir, f'{sim_type_label}_individual_{trait_name.replace("_avg", "")}_change.png')
        plt.savefig(plot_filepath)
        plt.close()
        with open(report_file_path, 'a', encoding='utf-8') as f:
            f.write(f"** 개별 시뮬레이션별 {trait_name.replace('_avg', '')} 평균 변화 그래프: {os.path.basename(plot_filepath)}\n\n")


        # 모든 시뮬레이션의 평균 형질 변화 (총 경향성)
        avg_prey_traits = np.zeros(max_total_turns + 1)
        avg_predator_traits = np.zeros(max_total_turns + 1)
        num_prey_data_points = np.zeros(max_total_turns + 1)
        num_predator_data_points = np.zeros(max_total_turns + 1)

        for results in sim_results_list:
            for t, data in results['data'].items():
                if t <= max_total_turns and data.get('prey_count', 0) > 0:
                    avg_prey_traits[t] += data['prey_traits'].get(trait_name, 0)
                    num_prey_data_points[t] += 1
                if t <= max_total_turns and data.get('predator_count', 0) > 0:
                    avg_predator_traits[t] += data['predator_traits'].get(trait_name, 0)
                    num_predator_data_points[t] += 1

        avg_prey_traits = np.divide(avg_prey_traits, num_prey_data_points, where=num_prey_data_points!=0)
        avg_predator_traits = np.divide(avg_predator_traits, num_predator_data_points, where=num_predator_data_points!=0)

        plt.figure(figsize=(12, 6))
        plt.plot(range(max_total_turns + 1), avg_prey_traits, label=f'평균 피식자 {trait_name.replace("_avg", "")}', color='green', linewidth=2)
        plt.plot(range(max_total_turns + 1), avg_predator_traits, label=f'평균 포식자 {trait_name.replace("_avg", "")}', color='red', linewidth=2)
        plt.title(f'{sim_type_label} 시나리오 - 모든 시뮬레이션의 평균 {trait_name.replace("_avg", "")} 변화')
        plt.xlabel('턴')
        plt.ylabel(f'평균 {trait_name.replace("_avg", "")}')
        plt.legend()
        plt.grid(True)
        plot_filepath = os.path.join(output_dir, f'{sim_type_label}_average_{trait_name.replace("_avg", "")}_change.png')
        plt.savefig(plot_filepath)
        plt.close()
        with open(report_file_path, 'a', encoding='utf-8') as f:
            f.write(f"** 모든 시뮬레이션의 평균 {trait_name.replace('_avg', '')} 변화 그래프: {os.path.basename(plot_filepath)}\n\n")


    # 색상 분포 변화 (개별 및 평균)
    for entity_type, label_prefix in [('prey_traits', '피식자'), ('predator_traits', '포식자')]:
        # 개별 시뮬레이션별 색상 분포
        for i, results in enumerate(sim_results_list):
            plt.figure(figsize=(12, 6))
            turns = sorted(results['data'].keys())

            current_colors_in_sim = sorted(list(set(l for t_data in results['data'].values()
                                                     for l in t_data[entity_type].get('color_dist', {}).keys())))

            if not turns or not current_colors_in_sim:
                plt.close()
                continue

            for color_idx, color_name in enumerate(color_names):
                if color_name not in current_colors_in_sim:
                    continue
                color_proportions = []
                for t in turns:
                    data_at_turn = results['data'].get(t, {})
                    entity_traits = data_at_turn.get(entity_type, {})
                    color_dist = entity_traits.get('color_dist', {})
                    total_entities_at_turn = data_at_turn.get('prey_count', 0) if entity_type == 'prey_traits' else data_at_turn.get('predator_count', 0)

                    prop = 0.0
                    if total_entities_at_turn > 0:
                        prop = color_dist.get(color_name, 0) / total_entities_at_turn
                    color_proportions.append(prop)

                if len(turns) == len(color_proportions) and len(color_proportions) > 0:
                    plot_color = ['red', 'blue', 'purple'][color_names.index(color_name)]
                    plt.plot(turns, color_proportions, label=f'시뮬 {i+1} {color_name} 비율', alpha=0.5, color=plot_color)
                else:
                    print(f"경고: 시뮬 {i+1} {label_prefix} 색상 {color_name} 그래프를 건너뜀. 데이터 길이 불일치/없음. (Turns:{len(turns)}, Props:{len(color_proportions)})")

            plt.title(f'시뮬레이션 {i+1}회 - {label_prefix} 색상 분포 변화')
            plt.xlabel('턴')
            plt.ylabel('개체 비율')
            plt.legend()
            plt.grid(True)
            plot_filepath = os.path.join(output_dir, f'{sim_type_label}_sim{i+1}_{label_prefix.lower()}_color_change.png')
            plt.savefig(plot_filepath)
            plt.close()
            with open(report_file_path, 'a', encoding='utf-8') as f:
                f.write(f"** 시뮬레이션 {i+1}회 {label_prefix} 색상 분포 그래프: {os.path.basename(plot_filepath)}\n\n")

        # 모든 시뮬레이션의 평균 색상 분포
        plt.figure(figsize=(12, 6))
        for color_idx, color_name in enumerate(color_names):
            avg_color_dist = np.zeros(max_total_turns + 1)
            num_color_data_points = np.zeros(max_total_turns + 1)

            for results in sim_results_list:
                for t, data in results['data'].items():
                    if t <= max_total_turns and (data.get('prey_count', 0) > 0 if entity_type == 'prey_traits' else data.get('predator_count', 0) > 0):
                        total_entities_at_turn = sum(data[entity_type].get('color_dist', {}).values())
                        if total_entities_at_turn > 0:
                            avg_color_dist[t] += data[entity_type].get('color_dist', {}).get(color_name, 0) / total_entities_at_turn
                            num_color_data_points[t] += 1

            avg_color_dist = np.divide(avg_color_dist, num_color_data_points, where=num_color_data_points!=0)
            plot_color = ['red', 'blue', 'purple'][color_idx]
            plt.plot(range(max_total_turns + 1), avg_color_dist, label=f'{label_prefix} {color_name} 비율', color=plot_color)

        plt.title(f'{sim_type_label} 시나리오 - 모든 시뮬레이션의 평균 {label_prefix} 색상 분포 변화')
        plt.xlabel('턴')
        plt.ylabel('개체 비율')
        plt.legend()
        plt.grid(True)
        plot_filepath = os.path.join(output_dir, f'{sim_type_label}_average_{label_prefix.lower()}_color_change_combined.png')
        plt.savefig(plot_filepath)
        plt.close()
        with open(report_file_path, 'a', encoding='utf-8') as f:
            f.write(f"** 모든 시뮬레이션의 평균 {label_prefix} 색상 분포 변화 그래프: {os.path.basename(plot_filepath)}\n\n")


    # 자원 선호도 분포 변화 (개별 및 평균)
    for entity_type, label_prefix in [('prey_traits', '피식자'), ('predator_traits', '포식자')]:
        # 개별 시뮬레이션별 자원 선호도 분포
        for i, results in enumerate(sim_results_list):
            plt.figure(figsize=(12, 6))
            turns = sorted(results['data'].keys())

            current_resource_prefs_in_sim = sorted(list(set(l for t_data in results['data'].values()
                                                             for l in t_data[entity_type].get('resource_pref_dist', {}).keys())))

            if not turns or not current_resource_prefs_in_sim:
                plt.close()
                continue

            for pref_name_idx, pref_name in enumerate(resource_pref_names):
                if pref_name not in current_resource_prefs_in_sim:
                    continue
                pref_proportions = []
                for t in turns:
                    data_at_turn = results['data'].get(t, {})
                    entity_traits = data_at_turn.get(entity_type, {})
                    resource_pref_dist = entity_traits.get('resource_pref_dist', {})
                    total_entities_at_turn = data_at_turn.get('prey_count', 0) if entity_type == 'prey_traits' else data_at_turn.get('predator_count', 0)

                    prop = 0.0
                    if total_entities_at_turn > 0:
                        prop = resource_pref_dist.get(pref_name, 0) / total_entities_at_turn
                    pref_proportions.append(prop)

                if len(turns) == len(pref_proportions) and len(pref_proportions) > 0:
                    plot_color = plt.colormaps['tab10'](pref_name_idx % plt.colormaps['tab10'].N)
                    plt.plot(turns, pref_proportions, label=f'시뮬 {i+1} {pref_name} 선호 비율', alpha=0.5, color=plot_color)
                else:
                    print(f"경고: 시뮬 {i+1} {label_prefix} 자원 선호도 {pref_name} 그래프를 건너뜀. 데이터 길이 불일치/없음. (Turns:{len(turns)}, Props:{len(pref_proportions)})")

            plt.title(f'시뮬레이션 {i+1}회 - {label_prefix} 자원 선호도 분포 변화')
            plt.xlabel('턴')
            plt.ylabel('개체 비율')
            plt.legend()
            plt.grid(True)
            plot_filepath = os.path.join(output_dir, f'{sim_type_label}_sim{i+1}_{label_prefix.lower()}_resource_pref_change.png')
            plt.savefig(plot_filepath)
            plt.close()
            with open(report_file_path, 'a', encoding='utf-8') as f:
                f.write(f"** 시뮬레이션 {i+1}회 {label_prefix} 자원 선호도 분포 그래프: {os.path.basename(plot_filepath)}\n\n")

        # 모든 시뮬레이션의 평균 자원 선호도 분포
        plt.figure(figsize=(12, 6))
        for pref_idx, pref_name in enumerate(resource_pref_names):
            avg_pref_dist = np.zeros(max_total_turns + 1)
            num_pref_data_points = np.zeros(max_total_turns + 1)

            for results in sim_results_list:
                for t, data in results['data'].items():
                    if t <= max_total_turns and (data.get('prey_count', 0) > 0 if entity_type == 'prey_traits' else data.get('predator_count', 0) > 0):
                        total_entities_at_turn = sum(data[entity_type].get('resource_pref_dist', {}).values())
                        if total_entities_at_turn > 0:
                            avg_pref_dist[t] += data[entity_type].get('resource_pref_dist', {}).get(pref_name, 0) / total_entities_at_turn
                            num_pref_data_points[t] += 1

            avg_pref_dist = np.divide(avg_pref_dist, num_pref_data_points, where=num_pref_data_points!=0)
            plot_color = plt.colormaps['tab10'](pref_idx % plt.colormaps['tab10'].N)
            plt.plot(range(max_total_turns + 1), avg_pref_dist, label=f'{label_prefix} {pref_name} 선호 비율', color=plot_color)

        plt.title(f'{sim_type_label} 시나리오 - 모든 시뮬레이션의 평균 {label_prefix} 자원 선호도 분포 변화')
        plt.xlabel('턴')
        plt.ylabel('개체 비율')
        plt.legend()
        plt.grid(True)
        plot_filepath = os.path.join(output_dir, f'{sim_type_label}_average_{label_prefix.lower()}_resource_pref_change_combined.png')
        plt.savefig(plot_filepath)
        plt.close()
        with open(report_file_path, 'a', encoding='utf-8') as f:
            f.write(f"** 모든 시뮬레이션의 평균 {label_prefix} 자원 선호도 분포 변화 그래프: {os.path.basename(plot_filepath)}\n\n")


    # 계통 분포 변화 (개별 및 평균)
    all_lineage_labels_overall = sorted(list(set(l for res in sim_results_list for t_data in res['data'].values()
                                                 for l in t_data['prey_traits'].get('lineage_dist', {}).keys() | t_data['predator_traits'].get('lineage_dist', {}).keys())))

    if all_lineage_labels_overall:
        lineage_color_map = {label: plt.colormaps['tab10'](i % plt.colormaps['tab10'].N)
                             for i, label in enumerate(all_lineage_labels_overall)}
    else:
        lineage_color_map = {}

    for entity_type, label_prefix in [('prey_traits', '피식자'), ('predator_traits', '포식자')]:
        # 개별 시뮬레이션별 계통 분포
        for i, results in enumerate(sim_results_list):
            plt.figure(figsize=(12, 6))
            turns = sorted(results['data'].keys())

            current_lineages_in_sim = sorted(list(set(l for t_data in results['data'].values()
                                                       for l in t_data[entity_type].get('lineage_dist', {}).keys())))

            if not turns or not current_lineages_in_sim:
                plt.close()
                continue

            for lineage_name in current_lineages_in_sim:
                lineage_proportions = []
                for t in turns:
                    data_at_turn = results['data'].get(t, {})
                    entity_traits = data_at_turn.get(entity_type, {})
                    lineage_dist = entity_traits.get('lineage_dist', {})
                    total_entities_at_turn = data_at_turn.get('prey_count', 0) if entity_type == 'prey_traits' else data_at_turn.get('predator_count', 0)

                    prop = 0.0
                    if total_entities_at_turn > 0:
                        prop = lineage_dist.get(lineage_name, 0) / total_entities_at_turn
                    lineage_proportions.append(prop)

                if len(turns) == len(lineage_proportions) and len(lineage_proportions) > 0:
                    plt.plot(turns, lineage_proportions, label=f'계통 {lineage_name}', color=lineage_color_map.get(lineage_name, 'black'))
                else:
                    print(f"경고: 시뮬 {i+1} {label_prefix} 계통 {lineage_name} 그래프를 건너뜀. 데이터 길이 불일치/없음. (Turns:{len(turns)}, Props:{len(lineage_proportions)})")

            plt.title(f'시뮬레이션 {i+1}회 - {label_prefix} 계통 분포 변화')
            plt.xlabel('턴')
            plt.ylabel('개체 비율')
            plt.legend()
            plt.grid(True)
            plot_filepath = os.path.join(output_dir, f'{sim_type_label}_sim{i+1}_{label_prefix.lower()}_lineage_change.png')
            plt.savefig(plot_filepath)
            plt.close()
            with open(report_file_path, 'a', encoding='utf-8') as f:
                f.write(f"** 시뮬레이션 {i+1}회 {label_prefix} 계통 분포 그래프: {os.path.basename(plot_filepath)}\n\n")

        # 모든 시뮬레이션의 평균 계통 분포
        plt.figure(figsize=(12, 6))
        for lineage_idx, lineage_name in enumerate(all_lineage_labels_overall):
            avg_lineage_dist = np.zeros(max_total_turns + 1)
            num_lineage_data_points = np.zeros(max_total_turns + 1)

            for results in sim_results_list:
                for t, data in results['data'].items():
                    if t <= max_total_turns and (data.get('prey_count', 0) > 0 if entity_type == 'prey_traits' else data.get('predator_count', 0) > 0):
                        total_entities_at_turn = sum(data[entity_type].get('lineage_dist', {}).values())
                        if total_entities_at_turn > 0:
                            avg_lineage_dist[t] += data[entity_type].get('lineage_dist', {}).get(lineage_name, 0) / total_entities_at_turn
                            num_lineage_data_points[t] += 1

            avg_lineage_dist = np.divide(avg_lineage_dist, num_lineage_data_points, where=num_lineage_data_points!=0)
            plt.plot(range(max_total_turns + 1), avg_lineage_dist, label=f'{label_prefix} 계통 {lineage_name} 비율', color=lineage_color_map.get(lineage_name, 'black'))

        plt.title(f'{sim_type_label} 시나리오 - 모든 시뮬레이션의 평균 {label_prefix} 계통 분포 변화')
        plt.xlabel('턴')
        plt.ylabel('개체 비율')
        plt.legend()
        plt.grid(True)
        plot_filepath = os.path.join(output_dir, f'{sim_type_label}_average_{label_prefix.lower()}_lineage_change_combined.png')
        plt.savefig(plot_filepath)
        plt.close()
        with open(report_file_path, 'a', encoding='utf-8') as f:
            f.write(f"** 모든 시뮬레이션의 평균 {label_prefix} 계통 분포 변화 그래프: {os.path.basename(plot_filepath)}\n\n")

    # 짝짓기 선호 색상 분포 변화
    for entity_type, label_prefix in [('prey_traits', '피식자'), ('predator_traits', '포식자')]:
        if entity_type == 'prey_traits':
            pref_dist_key = 'mate_pref_color_dist'
            pref_names_list = mate_pref_color_types
        else: # predator_traits
            pref_dist_key = 'mate_pref_color_dist'
            pref_names_list = mate_pref_color_types

        # 개별 시뮬레이션별 짝짓기 선호 색상 분포
        for i, results in enumerate(sim_results_list):
            plt.figure(figsize=(12, 6))
            turns = sorted(results['data'].keys())

            current_prefs_in_sim = sorted(list(set(l for t_data in results['data'].values()
                                                     for l in t_data[entity_type].get(pref_dist_key, {}).keys())))

            if not turns or not current_prefs_in_sim:
                plt.close()
                continue

            for pref_name_idx, pref_name in enumerate(pref_names_list):
                if pref_name not in current_prefs_in_sim:
                    continue
                pref_proportions = []
                for t in turns:
                    data_at_turn = results['data'].get(t, {})
                    entity_traits = data_at_turn.get(entity_type, {})
                    pref_dist = entity_traits.get(pref_dist_key, {})
                    total_entities_at_turn = data_at_turn.get('prey_count', 0) if entity_type == 'prey_traits' else data_at_turn.get('predator_count', 0)

                    prop = 0.0
                    if total_entities_at_turn > 0:
                        prop = pref_dist.get(pref_name, 0) / total_entities_at_turn
                    pref_proportions.append(prop)

                if len(turns) == len(pref_proportions) and len(pref_proportions) > 0:
                    plot_color = plt.colormaps['tab10'](pref_name_idx % plt.colormaps['tab10'].N)
                    plt.plot(turns, pref_proportions, label=f'시뮬 {i+1} {pref_name} 선호 비율', alpha=0.5, color=plot_color)
                else:
                    print(f"경고: 시뮬 {i+1} {label_prefix} 짝짓기 선호 색상 {pref_name} 그래프를 건너뜀. 데이터 길이 불일치/없음. (Turns:{len(turns)}, Props:{len(pref_proportions)})")

            plt.title(f'시뮬레이션 {i+1}회 - {label_prefix} 짝짓기 선호 색상 분포 변화')
            plt.xlabel('턴')
            plt.ylabel('개체 비율')
            plt.legend()
            plt.grid(True)
            plot_filepath = os.path.join(output_dir, f'{sim_type_label}_sim{i+1}_{label_prefix.lower()}_mate_pref_color_change.png')
            plt.savefig(plot_filepath)
            plt.close()
            with open(report_file_path, 'a', encoding='utf-8') as f:
                f.write(f"** 시뮬레이션 {i+1}회 {label_prefix} 짝짓기 선호 색상 분포 그래프: {os.path.basename(plot_filepath)}\n\n")

        # 모든 시뮬레이션의 평균 짝짓기 선호 색상 분포
        plt.figure(figsize=(12, 6))
        for pref_idx, pref_name in enumerate(pref_names_list):
            avg_pref_dist = np.zeros(max_total_turns + 1)
            num_pref_data_points = np.zeros(max_total_turns + 1)

            for results in sim_results_list:
                for t, data in results['data'].items():
                    if t <= max_total_turns and (data.get('prey_count', 0) > 0 if entity_type == 'prey_traits' else data.get('predator_count', 0) > 0):
                        total_entities_at_turn = sum(data[entity_type].get(pref_dist_key, {}).values())
                        if total_entities_at_turn > 0:
                            avg_pref_dist[t] += data[entity_type].get(pref_dist_key, {}).get(pref_name, 0) / total_entities_at_turn
                            num_pref_data_points[t] += 1

            avg_pref_dist = np.divide(avg_pref_dist, num_pref_data_points, where=num_pref_data_points!=0)
            plot_color = plt.colormaps['tab10'](pref_idx % plt.colormaps['tab10'].N)
            plt.plot(range(max_total_turns + 1), avg_pref_dist, label=f'{label_prefix} {pref_name} 선호 비율', color=plot_color)

        plt.title(f'{sim_type_label} 시나리오 - 모든 시뮬레이션의 평균 {label_prefix} 짝짓기 선호 색상 분포 변화')
        plt.xlabel('턴')
        plt.ylabel('개체 비율')
        plt.legend()
        plt.grid(True)
        plot_filepath = os.path.join(output_dir, f'{sim_type_label}_average_{label_prefix.lower()}_mate_pref_color_change_combined.png')
        plt.savefig(plot_filepath)
        plt.close()
        with open(report_file_path, 'a', encoding='utf-8') as f:
            f.write(f"** 모든 시뮬레이션의 평균 {label_prefix} 짝짓기 선호 색상 분포 변화 그래프: {os.path.basename(plot_filepath)}\n\n")

    # 짝짓기 선호 계통 분포 변화
    for entity_type, label_prefix in [('prey_traits', '피식자'), ('predator_traits', '포식자')]:
        if entity_type == 'prey_traits':
            pref_dist_key = 'mate_pref_lineage_dist'
            pref_names_list = mate_pref_lineage_types
        else: # predator_traits
            pref_dist_key = 'mate_pref_lineage_dist'
            pref_names_list = mate_pref_lineage_types

        # 개별 시뮬레이션별 짝짓기 선호 계통 분포
        for i, results in enumerate(sim_results_list):
            plt.figure(figsize=(12, 6))
            turns = sorted(results['data'].keys())

            current_prefs_in_sim = sorted(list(set(l for t_data in results['data'].values()
                                                     for l in t_data[entity_type].get(pref_dist_key, {}).keys())))

            if not turns or not current_prefs_in_sim:
                plt.close()
                continue

            for pref_name_idx, pref_name in enumerate(pref_names_list):
                if pref_name not in current_prefs_in_sim:
                    continue
                pref_proportions = []
                for t in turns:
                    data_at_turn = results['data'].get(t, {})
                    entity_traits = data_at_turn.get(entity_type, {})
                    pref_dist = entity_traits.get(pref_dist_key, {})
                    total_entities_at_turn = data_at_turn.get('prey_count', 0) if entity_type == 'prey_traits' else data_at_turn.get('predator_count', 0)

                    prop = 0.0
                    if total_entities_at_turn > 0:
                        prop = pref_dist.get(pref_name, 0) / total_entities_at_turn
                    pref_proportions.append(prop)

                if len(turns) == len(pref_proportions) and len(pref_proportions) > 0:
                    plot_color = plt.colormaps['tab10'](pref_name_idx % plt.colormaps['tab10'].N)
                    plt.plot(turns, pref_proportions, label=f'시뮬 {i+1} {pref_name} 선호 비율', alpha=0.5, color=plot_color)
                else:
                    print(f"경고: 시뮬 {i+1} {label_prefix} 짝짓기 선호 계통 {pref_name} 그래프를 건너뜀. 데이터 길이 불일치/없음. (Turns:{len(turns)}, Props:{len(pref_proportions)})")

            plt.title(f'시뮬레이션 {i+1}회 - {label_prefix} 짝짓기 선호 계통 분포 변화')
            plt.xlabel('턴')
            plt.ylabel('개체 비율')
            plt.legend()
            plt.grid(True)
            plot_filepath = os.path.join(output_dir, f'{sim_type_label}_sim{i+1}_{label_prefix.lower()}_mate_pref_lineage_change.png')
            plt.savefig(plot_filepath)
            plt.close()
            with open(report_file_path, 'a', encoding='utf-8') as f:
                f.write(f"** 시뮬레이션 {i+1}회 {label_prefix} 짝짓기 선호 계통 분포 그래프: {os.path.basename(plot_filepath)}\n\n")

        # 모든 시뮬레이션의 평균 짝짓기 선호 계통 분포
        plt.figure(figsize=(12, 6))
        for pref_idx, pref_name in enumerate(pref_names_list):
            avg_pref_dist = np.zeros(max_total_turns + 1)
            num_pref_data_points = np.zeros(max_total_turns + 1)

            for results in sim_results_list:
                for t, data in results['data'].items():
                    if t <= max_total_turns and (data.get('prey_count', 0) > 0 if entity_type == 'prey_traits' else data.get('predator_count', 0) > 0):
                        total_entities_at_turn = sum(data[entity_type].get(pref_dist_key, {}).values())
                        if total_entities_at_turn > 0:
                            avg_pref_dist[t] += data[entity_type].get(pref_dist_key, {}).get(pref_name, 0) / total_entities_at_turn
                            num_pref_data_points[t] += 1

            avg_pref_dist = np.divide(avg_pref_dist, num_pref_data_points, where=num_pref_data_points!=0)
            plot_color = plt.colormaps['tab10'](pref_idx % plt.colormaps['tab10'].N)
            plt.plot(range(max_total_turns + 1), avg_pref_dist, label=f'{label_prefix} {pref_name} 선호 비율', color=plot_color)

        plt.title(f'{sim_type_label} 시나리오 - 모든 시뮬레이션의 평균 {label_prefix} 짝짓기 선호 계통 분포 변화')
        plt.xlabel('턴')
        plt.ylabel('개체 비율')
        plt.legend()
        plt.grid(True)
        plot_filepath = os.path.join(output_dir, f'{sim_type_label}_average_{label_prefix.lower()}_mate_pref_lineage_change_combined.png')
        plt.savefig(plot_filepath)
        plt.close()
        with open(report_file_path, 'a', encoding='utf-8') as f:
            f.write(f"** 모든 시뮬레이션의 평균 {label_prefix} 짝짓기 선호 계통 분포 변화 그래프: {os.path.basename(plot_filepath)}\n\n")

    # 자원 이용 전략 분포 변화 (피식자)
    for entity_type, label_prefix in [('prey_traits', '피식자')]: # 피식자만 해당
        pref_dist_key = 'foraging_strategy_dist'
        pref_names_list = foraging_strategy_names

        # 개별 시뮬레이션별 자원 이용 전략 분포
        for i, results in enumerate(sim_results_list):
            plt.figure(figsize=(12, 6))
            turns = sorted(results['data'].keys())

            current_prefs_in_sim = sorted(list(set(l for t_data in results['data'].values()
                                                     for l in t_data[entity_type].get(pref_dist_key, {}).keys())))

            if not turns or not current_prefs_in_sim:
                plt.close()
                continue

            for pref_name_idx, pref_name in enumerate(pref_names_list):
                if pref_name not in current_prefs_in_sim:
                    continue
                pref_proportions = []
                for t in turns:
                    data_at_turn = results['data'].get(t, {})
                    entity_traits = data_at_turn.get(entity_type, {})
                    pref_dist = entity_traits.get(pref_dist_key, {})
                    total_entities_at_turn = data_at_turn.get('prey_count', 0)

                    prop = 0.0
                    if total_entities_at_turn > 0:
                        prop = pref_dist.get(pref_name, 0) / total_entities_at_turn
                    pref_proportions.append(prop)

                if len(turns) == len(pref_proportions) and len(pref_proportions) > 0:
                    plot_color = plt.colormaps['tab10'](pref_name_idx % plt.colormaps['tab10'].N)
                    plt.plot(turns, pref_proportions, label=f'시뮬 {i+1} {pref_name} 전략', alpha=0.5, color=plot_color)
                else:
                    print(f"경고: 시뮬 {i+1} {label_prefix} 자원 이용 전략 {pref_name} 그래프를 건너뜀. 데이터 길이 불일치/없음. (Turns:{len(turns)}, Props:{len(pref_proportions)})")

            plt.title(f'시뮬레이션 {i+1}회 - {label_prefix} 자원 이용 전략 분포 변화')
            plt.xlabel('턴')
            plt.ylabel('개체 비율')
            plt.legend()
            plt.grid(True)
            plot_filepath = os.path.join(output_dir, f'{sim_type_label}_sim{i+1}_{label_prefix.lower()}_foraging_strategy_change.png')
            plt.savefig(plot_filepath)
            plt.close()
            with open(report_file_path, 'a', encoding='utf-8') as f:
                f.write(f"** 시뮬레이션 {i+1}회 {label_prefix} 자원 이용 전략 분포 그래프: {os.path.basename(plot_filepath)}\n\n")

        # 모든 시뮬레이션의 평균 자원 이용 전략 분포
        plt.figure(figsize=(12, 6))
        for pref_idx, pref_name in enumerate(pref_names_list):
            avg_pref_dist = np.zeros(max_total_turns + 1)
            num_pref_data_points = np.zeros(max_total_turns + 1)

            for results in sim_results_list:
                for t, data in results['data'].items():
                    if t <= max_total_turns and data.get('prey_count', 0) > 0:
                        total_entities_at_turn = sum(data[entity_type].get(pref_dist_key, {}).values())
                        if total_entities_at_turn > 0:
                            avg_pref_dist[t] += data[entity_type].get(pref_dist_key, {}).get(pref_name, 0) / total_entities_at_turn
                            num_pref_data_points[t] += 1

            avg_pref_dist = np.divide(avg_pref_dist, num_pref_data_points, where=num_pref_data_points!=0)
            plot_color = plt.colormaps['tab10'](pref_idx % plt.colormaps['tab10'].N)
            plt.plot(range(max_total_turns + 1), avg_pref_dist, label=f'{label_prefix} {pref_name} 전략 비율', color=plot_color)

        plt.title(f'{sim_type_label} 시나리오 - 모든 시뮬레이션의 평균 {label_prefix} 자원 이용 전략 분포 변화')
        plt.xlabel('턴')
        plt.ylabel('개체 비율')
        plt.legend()
        plt.grid(True)
        plot_filepath = os.path.join(output_dir, f'{sim_type_label}_average_{label_prefix.lower()}_foraging_strategy_change_combined.png')
        plt.savefig(plot_filepath)
        plt.close()
        with open(report_file_path, 'a', encoding='utf-8') as f:
            f.write(f"** 모든 시뮬레이션의 평균 {label_prefix} 자원 이용 전략 분포 변화 그래프: {os.path.basename(plot_filepath)}\n\n")

    # 사냥 전략 분포 변화 (포식자)
    for entity_type, label_prefix in [('predator_traits', '포식자')]: # 포식자만 해당
        pref_dist_key = 'hunting_strategy_dist'
        pref_names_list = hunting_strategy_names

        # 개별 시뮬레이션별 사냥 전략 분포
        for i, results in enumerate(sim_results_list):
            plt.figure(figsize=(12, 6))
            turns = sorted(results['data'].keys())

            current_prefs_in_sim = sorted(list(set(l for t_data in results['data'].values()
                                                     for l in t_data[entity_type].get(pref_dist_key, {}).keys())))

            if not turns or not current_prefs_in_sim:
                plt.close()
                continue

            for pref_name_idx, pref_name in enumerate(pref_names_list):
                if pref_name not in current_prefs_in_sim:
                    continue
                pref_proportions = []
                for t in turns:
                    data_at_turn = results['data'].get(t, {})
                    entity_traits = data_at_turn.get(entity_type, {})
                    pref_dist = entity_traits.get(pref_dist_key, {})
                    total_entities_at_turn = data_at_turn.get('predator_count', 0)

                    prop = 0.0
                    if total_entities_at_turn > 0:
                        prop = pref_dist.get(pref_name, 0) / total_entities_at_turn
                    pref_proportions.append(prop)

                if len(turns) == len(pref_proportions) and len(pref_proportions) > 0:
                    plot_color = plt.colormaps['tab10'](pref_name_idx % plt.colormaps['tab10'].N)
                    plt.plot(turns, pref_proportions, label=f'시뮬 {i+1} {pref_name} 전략', alpha=0.5, color=plot_color)
                else:
                    print(f"경고: 시뮬 {i+1} {label_prefix} 사냥 전략 {pref_name} 그래프를 건너뜀. 데이터 길이 불일치/없음. (Turns:{len(turns)}, Props:{len(pref_proportions)})")

            plt.title(f'시뮬레이션 {i+1}회 - {label_prefix} 사냥 전략 분포 변화')
            plt.xlabel('턴')
            plt.ylabel('개체 비율')
            plt.legend()
            plt.grid(True)
            plot_filepath = os.path.join(output_dir, f'{sim_type_label}_sim{i+1}_{label_prefix.lower()}_hunting_strategy_change.png')
            plt.savefig(plot_filepath)
            plt.close()
            with open(report_file_path, 'a', encoding='utf-8') as f:
                f.write(f"** 시뮬레이션 {i+1}회 {label_prefix} 사냥 전략 분포 그래프: {os.path.basename(plot_filepath)}\n\n")

        # 모든 시뮬레이션의 평균 사냥 전략 분포
        plt.figure(figsize=(12, 6))
        for pref_idx, pref_name in enumerate(pref_names_list):
            avg_pref_dist = np.zeros(max_total_turns + 1)
            num_pref_data_points = np.zeros(max_total_turns + 1)

            for results in sim_results_list:
                for t, data in results['data'].items():
                    if t <= max_total_turns and data.get('predator_count', 0) > 0:
                        total_entities_at_turn = sum(data[entity_type].get(pref_dist_key, {}).values())
                        if total_entities_at_turn > 0:
                            avg_pref_dist[t] += data[entity_type].get(pref_dist_key, {}).get(pref_name, 0) / total_entities_at_turn
                            num_pref_data_points[t] += 1

            avg_pref_dist = np.divide(avg_pref_dist, num_pref_data_points, where=num_pref_data_points!=0)
            plot_color = plt.colormaps['tab10'](pref_idx % plt.colormaps['tab10'].N)
            plt.plot(range(max_total_turns + 1), avg_pref_dist, label=f'{label_prefix} {pref_name} 전략 비율', color=plot_color)

        plt.title(f'{sim_type_label} 시나리오 - 모든 시뮬레이션의 평균 {label_prefix} 사냥 전략 분포 변화')
        plt.xlabel('턴')
        plt.ylabel('개체 비율')
        plt.legend()
        plt.grid(True)
        plot_filepath = os.path.join(output_dir, f'{sim_type_label}_average_{label_prefix.lower()}_hunting_strategy_change_combined.png')
        plt.savefig(plot_filepath)
        plt.close()
        with open(report_file_path, 'a', encoding='utf-8') as f:
            f.write(f"** 모든 시뮬레이션의 평균 {label_prefix} 사냥 전략 분포 변화 그래프: {os.path.basename(plot_filepath)}\n\n")


    # 턴별 로그를 모든 시뮬레이션에 대해 하나의 파일로 합쳐서 저장
    combined_turn_logs_filepath = os.path.join(output_dir, f"{sim_type_label}_combined_turn_logs.txt")
    with open(combined_turn_logs_filepath, 'w', encoding='utf-8') as f:
        f.write(f"### {sim_type_label} 시뮬레이션 통합 턴 로그 ###\n\n")
        for i, results in enumerate(sim_results_list):
            f.write(f"\n--- 시뮬레이션 {i+1}회 로그 (시뮬레이션 종료 턴: {results['final_turn']}) ---\n")
            turn_logs_for_sim = results.get('turn_logs', {})
            for turn in sorted(turn_logs_for_sim.keys()):
                f.write(f"---- 턴 {turn} ----\n")
                for log_entry in turn_logs_for_sim[turn]:
                    f.write(log_entry + "\n")
                f.write("\n")
            f.write("\n")

    with open(report_file_path, 'a', encoding='utf-8') as f:
        f.write(f"** 통합 턴 로그 파일: {os.path.basename(combined_turn_logs_filepath)}\n\n")

    # 4. 각 시뮬레이션별 결과 출력 (요약)
    with open(report_file_path, 'a', encoding='utf-8') as f:
        f.write("\n--- 각 시뮬레이션 요약 ---\n")
        for i, results in enumerate(sim_results_list):
            f.write(f"\n## 시뮬레이션 {i+1}:\n")
            f.write(f"  종료 턴: {results['final_turn']}\n")
            final_prey_count = results['data'].get(results['final_turn'], {}).get('prey_count', 0)
            final_predator_count = results['data'].get(results['final_turn'], {}).get('predator_count', 0)
            f.write(f"  최종 피식자 개체수: {final_prey_count}\n")
            f.write(f"  최종 포식자 개체수: {final_predator_count}\n")
            f.write(f"  피식자 멸종 여부: {'멸종' if not results['prey_alive'] else '생존'}\n")
            f.write(f"  포식자 멸종 여부: {'멸종' if not results['predator_alive'] else '생존'}\n")

    # 5. 모든 시뮬레이션에서의 총 경향성 (텍스트 요약)
    with open(report_file_path, 'a', encoding='utf-8') as f:
        f.write("\n--- 모든 시뮬레이션 총 경향성 ---\n")
        total_prey_extinctions = sum(1 for res in sim_results_list if not res['prey_alive'])
        total_predator_extinctions = sum(1 for res in sim_results_list if not res['predator_alive'])

        final_prey_counts = [res['data'][res['final_turn']]['prey_count'] for res in sim_results_list if res['prey_alive']]
        final_predator_counts = [res['data'][res['final_turn']]['predator_count'] for res in sim_results_list if res['predator_alive']]

        avg_final_prey_count = np.mean(final_prey_counts) if final_prey_counts else 0
        avg_final_predator_count = np.mean(final_predator_counts) if final_predator_counts else 0

        f.write(f"  총 {len(sim_results_list)}회 실행 중:\n")
        f.write(f"  피식자 멸종 횟수: {total_prey_extinctions}회\n")
        f.write(f"  포식자 멸종 횟수: {total_predator_extinctions}회\n")
        f.write(f"  평균 최종 피식자 개체수 (생존 시뮬레이션 기준): {avg_final_prey_count:.2f}\n")
        f.write(f"  평균 최종 포식자 개체수 (생존 시뮬레이션 기준): {avg_final_predator_count:.2f}\n")
        f.write("\n(형질 변화의 총 경향성은 위에 제시된 평균 형질 변화 그래프를 통해 비교 분석할 수 있습니다.)\n")

    print(f"\n모든 시뮬레이션 및 분석이 완료되었습니다. 생성된 폴더를 확인하세요.")


if __name__ == "__main__":
    # 시뮬레이션 공통 파라미터 설정 (SimulationSettings 클래스 활용)
    sim_common_params = {
        'width': SimulationSettings.WIDTH,
        'height': SimulationSettings.HEIGHT,
        'max_food_per_tile': SimulationSettings.MAX_FOOD_PER_TILE,
        'food_regen_rate': SimulationSettings.FOOD_REGEN_RATE,
        'initial_prey_count': SimulationSettings.INITIAL_PREY_COUNT,
        'initial_predator_count': SimulationSettings.INITIAL_PREDATOR_COUNT,
        'max_turns': SimulationSettings.MAX_TURNS
    }

    num_sims_to_run = 5 # 각 시나리오별로 실행할 시뮬레이션 횟수

    # 결과 보고서와 그래프를 저장할 메인 폴더 생성
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    results_base_dir = f"simulation_results_{timestamp}"
    os.makedirs(results_base_dir, exist_ok=True)

    # 유전자 기반 시뮬레이션 실행 및 결과 분석
    gene_based_output_dir = os.path.join(results_base_dir, "gene_based")
    os.makedirs(gene_based_output_dir, exist_ok=True)
    gene_based_results = run_multiple_simulations(num_sims_to_run, "유전자 기반", sim_common_params, base_seed=None)

    analyze_and_plot_results(gene_based_results, "유전자 기반", gene_based_output_dir, base_seed_used=None)

    # 무작위 시뮬레이션 실행 및 결과 분석
    random_output_dir = os.path.join(results_base_dir, "random")
    os.makedirs(random_output_dir, exist_ok=True)
    random_results = run_multiple_simulations(num_sims_to_run, "무작위", sim_common_params, base_seed=None)

    analyze_and_plot_results(random_results, "무작위", random_output_dir, base_seed_used=None)

    print("\n모든 시뮬레이션 및 분석이 완료되었습니다. 생성된 폴더를 확인하세요.")